# Centralized Tools

## Centralized Functions - JSON Files

In [1]:
import json
import re
from typing import Dict
from pathlib import Path
import textwrap
from IPython.display import display, Markdown
from utils.generic_utils import read_from_json_file


def clean_json_string(json_string: str):
    """
    Remove unwanted control characters from the JSON string.
    """
    return re.sub(r"[\x00-\x1F\x7F]", "", json_string)


def load_and_clean_json_file(json_file: Path | str) -> Dict:
    """
    Load a JSON file and decode all Unicode escape sequences.

    This function reads a JSON file, ensures it is a valid JSON string,
    and removes unwanted control characters before parsing it.

    Args:
        json_file (Path | str): The path to the JSON file.

    Returns:
        dict: The cleaned JSON data.

    Raises:
        FileNotFoundError: If the file does not exist.
        ValueError: If the JSON data is invalid or cannot be parsed.

    Example Usage:
    --------------
    >>> json_data = load_and_clean_json_file("config.json")
    >>> print(json_data["api_key"])
    """
    json_data = read_from_json_file(json_file)  # This could be Dict or Any

    if isinstance(json_data, dict):
        json_string = json.dumps(
            json_data, ensure_ascii=False
        )  # Convert dict to string
    else:
        json_string = str(json_data)  # Convert single value to string

    cleaned_json_string = clean_json_string(json_string)  # Ensure it's cleaned
    return json.loads(cleaned_json_string)  # Convert back to a dictionary


def format_json_readable(json_obj, indent=2, wrap_width=80) -> str:
    """
    Formats JSON data with indentation and wraps long text for better readability.

    Args:
        json_obj (dict | list): JSON object or list to format.
        indent (int): Number of spaces to use for indentation. Default is 2.
        wrap_width (int): Maximum width of text before wrapping. Default is 80.

    Returns:
        str: The formatted and wrapped JSON string.

    Example Usage:
    --------------
    >>> json_data = {"name": "Alice", "role": "Senior Data Scientist", "skills": ["Python", "ML", "NLP"]}
    >>> formatted_json = format_json_readable(json_data)
    >>> print(formatted_json)
    """
    formatted_json = json.dumps(json_obj, indent=indent, ensure_ascii=False)

    # Wrap long lines for better readability
    formatted_json = "\n".join(
        [
            textwrap.fill(line, width=wrap_width) if len(line) > wrap_width else line
            for line in formatted_json.split("\n")
        ]
    )

    return formatted_json


def display_json_pretty(json_input: dict | str | Path, wrap_width: int = 120):
    """
    Displays JSON data in a human-readable format using Markdown.

    This function takes a JSON dictionary, a JSON string, or a file path,
    formats it, and displays it in a scrollable Markdown code block.

    Args:
        json_input (dict | str | Path): JSON data (dict), JSON string, or JSON file path.
        wrap_width (int): Maximum width of text before wrapping. Default is 120.

    Raises:
        ValueError: If the JSON string is invalid or cannot be parsed.

    Example Usage:
    --------------
    Display JSON from a dictionary:
    >>> json_data = {"name": "Alice", "role": "Senior Data Scientist"}
    >>> display_json_pretty(json_data)

    Display JSON from a file:
    >>> display_json_pretty("config.json")
    """
    if isinstance(json_input, dict):
        data = json_input  # Use dictionary directly
    elif isinstance(json_input, Path) or (
        isinstance(json_input, str) and not json_input.strip().startswith(("{", "["))
    ):
        data = load_and_clean_json_file(str(json_input))  # Load JSON from file
    else:
        data = json.loads(json_input)  # Parse JSON string

    # Format JSON for readability
    formatted_json = format_json_readable(data, wrap_width=wrap_width)

    # Display with Markdown to prevent horizontal scrolling
    display(Markdown(f"```json\n{formatted_json}\n```"))

## Centralized Functions - DataFrames

In [2]:
from pathlib import Path
from typing import Tuple
import xlwings as xw
import pandas as pd
from typing import Set, Tuple, Any, Union
import numpy as np

from docx import Document


def load_excel_sheet(file_path: str | Path, sheet_name: str) -> Tuple:
    """Load an Excel sheet using xlwings."""
    file_path = Path(file_path) if isinstance(file_path, str) else file_path
    wb = xw.Book(file_path)
    sheet = wb.sheets[sheet_name]
    return wb, sheet


def get_underlined_cells(
    sheet: xw.Sheet, return_coords: bool = False
) -> Union[Tuple[pd.DataFrame, Set[Tuple[Any, int]]], pd.DataFrame]:
    """
    Extract values from underlined cells while keeping the first two columns as is.

    Returns:
    - DataFrame of all values (non-underlined = None except first 2 cols)
    - Set of (row_key, col_index) coordinates where underline was detected
    """
    table_range = sheet.range("A1").expand("table")
    underlined_cells = []
    underlined_coords = set()

    for i in range(1, table_range.rows.count + 1):
        underlined_row = []

        for j in range(1, table_range.columns.count + 1):
            cell = table_range.api.Cells(i, j)
            value = cell.Value

            if j <= 2:
                underlined_row.append(value)
            elif cell.Font.Underline != -4142:  # Underlined
                underlined_row.append(value)
                if i > 1:  # Skip header row
                    underlined_coords.add((table_range.api.Cells(i, 1).Value, j))
                # (row_key, col_index)
            else:
                underlined_row.append(None)

        underlined_cells.append(underlined_row)

    df = pd.DataFrame(underlined_cells)
    df.set_index(0, inplace=True)

    if return_coords:
        return df, underlined_coords
    else:
        return df


def clean_and_remove_rows(df):
    """Remove None values and reorganize the dataframe."""
    max_length = df.apply(lambda row: row.dropna().shape[0], axis=1).max()
    new_df = pd.DataFrame(index=df.index)

    for i in range(max_length):
        new_df[i] = df.apply(lambda row: get_ith_non_nan(row, i), axis=1)
    return new_df.iloc[2:]  # Remove the first two rows


def get_ith_non_nan(row, i):
    """Retrieve the ith non-NaN value from a row."""
    non_nan = row.dropna()

    return non_nan.iloc[i] if i < len(non_nan) else np.nan


def rename_dataframe_columns(df):
    """Rename index and columns dynamically based on the number of columns."""
    df.index.name = "responsibility_key"
    column_names = ["original_responsibility"] + [
        f"edited_responsibility_{i}" for i in range(1, df.shape[1])
    ]
    df.columns = column_names[: df.shape[1]]

    return df


def json_to_docx(json_data: dict, output_file: Path | str):
    doc = Document()

    if isinstance(json_data, str):
        data = json.loads(json_data)

    else:
        data = json_data

    for main_key, sub_dict in data.items():

        for key, value in sub_dict.items():
            doc.add_paragraph(f"{key}:")
            doc.add_paragraph(value)
            doc.add_paragraph("")  # Add blank line

    if isinstance(output_file, Path):
        doc.save(str(output_file))
    else:
        doc.save(output_file)


def get_underlined_col_names(df: pd.DataFrame, underlined_coords):
    """
    Given the DataFrame and set of (row_key, col_number) coordinates,
    return a list of (row_key, column_name) where the cell was underlined.
    """
    readable = []
    for row_key, col_num in underlined_coords:
        try:
            col_name = df.columns[col_num - 1]
            readable.append((row_key, col_name))
        except IndexError:
            print(f"⚠️ Column number {col_num} out of range for row {row_key}")
            continue
    return readable


def summarize_underlined_column_frequencies(
    df_all_data: pd.DataFrame, underlined_coords: Set[Tuple[Any, int]]
) -> pd.DataFrame:
    """
    Summarize how often each column appears in the set of underlined cell coordinates,
    and calculate its frequency both as a count and as a percentage of total underlined hits.

    Parameters:
    ----------
    df_all_data : pd.DataFrame
        The full DataFrame returned from `get_underlined_cells()`, including headers.

    underlined_coords : Set[Tuple[Any, int]]
        A set of (row_key, col_index) tuples representing underlined cell positions.
        Column indices are assumed to be 1-based Excel-style indices.

    Returns:
    -------
    pd.DataFrame
        A DataFrame with columns:
            - 'Column Name': the name of the column (from df_all_data.columns)
            - 'Underline Count': how many underlined cells were found in this column
            - 'Frequency (%)': percentage share of all underlined cells
        Sorted in descending order by underline count.
    """
    # Step 1: Frequency count by Excel-style column number
    freq_dict = {}
    for _, col_num in underlined_coords:
        freq_dict[col_num] = freq_dict.get(col_num, 0) + 1

    total_underlines = sum(freq_dict.values())

    # Step 2: Map to column names
    col_name_counts = {}
    for col_num, count in freq_dict.items():
        try:
            col_name = df_all_data.columns[col_num - 1]  # Adjust for 1-based indexing
            col_name_counts[col_name] = count
        except IndexError:
            print(
                f"⚠️ Column {col_num} out of range for DataFrame with {len(df_all_data.columns)} columns."
            )
            continue

    # Step 3: Build DataFrame
    summary_df = pd.DataFrame(
        [
            {
                "Column Name": col,
                "Underline Count": count,
                "Frequency (%)": round((count / total_underlines) * 100, 1),
            }
            for col, count in col_name_counts.items()
        ]
    ).sort_values("Underline Count", ascending=False)

    return summary_df

## Centralized File/Dir Path Imports

In [3]:
from project_config import (
    JOB_POSTING_URLS_FILE,
    JOB_POSTING_URLS_FILTERED_FILE,
    JOB_POSTING_URLS_TO_EXCLUDE_FILE,
    JOB_DESCRIPTIONS_JSON_FILE,
    JOB_REQUIREMENTS_JSON_FILE,
    mapping_file_name,
)

# Iteration 0 imports
from project_config import (
    ITERATE_0_OPENAI_DIR,
    REQS_FILES_ITERATE_0_OPENAI_DIR,
    RESPS_FILES_ITERATE_0_OPENAI_DIR,
    PRUNED_RESPS_FILES_ITERATE_0_OPENAI_DIR,
    SIMILARITY_METRICS_ITERATE_0_OPENAI_DIR,
    ITERATE_0_ANTHROPIC_DIR,
    REQS_FILES_ITERATE_0_ANTHROPIC_DIR,
    RESPS_FILES_ITERATE_0_ANTHROPIC_DIR,
    PRUNED_RESPS_FILES_ITERATE_0_ANTHROPIC_DIR,
    SIMILARITY_METRICS_ITERATE_0_ANTHROPIC_DIR,
    URL_TO_FILE_MAPPING_FILE_ITERATE_0_ANTHROPIC,
)

# Iterate 1 imports
from project_config import (
    ITERATE_1_OPENAI_DIR,
    REQS_FILES_ITERATE_1_OPENAI_DIR,
    RESPS_FILES_ITERATE_1_OPENAI_DIR,
    PRUNED_RESPS_FILES_ITERATE_1_DIR_OPENAI,
    SIMILARITY_METRICS_ITERATE_1_OPENAI_DIR,
    url_to_file_mapping_file_iterate_1_openai,
    ITERATE_1_ANTHROPIC_DIR,
    REQS_FILES_ITERATE_1_ANTHROPIC_DIR,
    RESPS_FILES_ITERATE_1_ANTHROPIC_DIR,
    PRUNED_RESPS_FILES_ITERATE_1_ANTHROPIC_DIR,
    SIMILARITY_METRICS_ITERATE_1_ANTHROPIC_DIR,
    URL_TO_FILE_MAPPING_FILE_ITERATE_1_ANTHROPIC,
)

from project_config import RESPS_REQS_MATCHINGS_DIR, REVIEWED_MATCHINGS_DIR

# Check & Edit Resume Files by Posting

## Accenture Job Posting

In [6]:
from evaluation_optimization.create_mapping_file import load_mappings_model_from_json
from models.resume_job_description_io_models import JobFileMappings


accenture_file = r"C:\github\job_bot\input_output\human_review\resps_reqs_matching\reviewed_matchings\Accenture_Enterprise_AI_Value_Strategy_Senior_Manager_crosstab.xlsx"

mapping_file = ITERATE_1_OPENAI_DIR / mapping_file_name

mapping_model = load_mappings_model_from_json(mapping_file)

print(type(mapping_model.root))  # Should print <class 'dict'>

# acn_url = next(
#     (
#         url
#         for url, paths in mapping_model.root.items()
#         if any("accenture" in str(value).lower() for value in vars(paths).values())
#     ),
#     None,
# )

acn_url = next(
    (url for url in mapping_model.root.keys() if "accenture" in str(url).lower()),
    None,
)
print(acn_url)

2025-03-21 15:11:44,489 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\evaluation_optimization\evaluation_optimization_by_openai\iteration_1\url_to_file_mapping.json
2025-03-21 15:11:44,490 - evaluation_optimization.create_mapping_file - INFO - Loaded and validated mapping file from C:\github\job_bot\input_output\evaluation_optimization\evaluation_optimization_by_openai\iteration_1\url_to_file_mapping.json


<class 'dict'>
https://www.accenture.com/us-en/careers/jobdetails?id=R00251798_en&src=LINKEDINJP


#### Dataframe format

In [10]:
# Use this to help select which edited text to use for each responsibility

file_path = r"C:\github\job_bot\input_output\human_review\resps_reqs_matching\reviewed_matchings\Accenture_Enterprise_AI_Value_Strategy_Senior_Manager_crosstab.xlsx"
sheet_name = "Crosstab"


wb, sheet = load_excel_sheet(file_path, sheet_name)
df = get_underlined_cells(sheet)


cleaned_df = clean_and_remove_rows(df)
final_df = rename_dataframe_columns(cleaned_df)

pd.set_option("display.max_columns", None)

display(final_df)

wb.close()

,original_responsibility,edited_responsibility_1
responsibility_key,,
0.responsibilities.0,Conversational AI & NLP: Engineered the core d...,"Engineered and executed AI-first strategies, f..."
0.responsibilities.1,Thought Generation & Processing: Designed pipe...,Thought Generation & Processing: Developed adv...
0.responsibilities.2,Automated Evaluation & Adaptation: Developed a...,Developed an AI-based system to assess user in...
0.responsibilities.3,State & Topic Management: Built a stateful tra...,Built a stateful AI system to optimize convers...
0.responsibilities.4,Asynchronous AI Integration: Optimized API per...,Asynchronous AI Integration: Led enhancements ...
1.responsibilities.0,Provided strategic insights to a major global ...,Optimized the service partner ecosystem for a ...
1.responsibilities.1,Assisted a U.S.-based international services p...,Assisted a U.S.-based international services p...
1.responsibilities.2,Co-authored an industry-recognized report on M...,Co-authored a seminal report on M&A in the eng...
1.responsibilities.3,Enhanced data quality and consistency by integ...,Enhanced data quality and consistency through ...


#### JSON format: Original

In [ ]:
file_path = r"C:\github\job_bot\input_output\human_review\resps_reqs_matching\reviewed_matchings\Accenture_Enterprise_AI_Value_Strategy_Senior_Manager_crosstab.xlsx"
sheet_name = "Crosstab"

wb, sheet = load_excel_sheet(file_path, sheet_name)
df = get_underlined_cells(sheet)

cleaned_df = clean_and_remove_rows(df)
final_df = rename_dataframe_columns(cleaned_df)
final_df.drop("edited_responsibility_1", axis=1, inplace=True)


# print(final_df)
json_string = final_df.to_json()
json_data = json.loads(json_string)  # Convert string to dictionary

print("Original:")


# Format JSON for readability
formatted_json = format_json_readable(json_data, wrap_width=120)

# Display with Markdown to prevent horizontal scrolling
display(Markdown(f"```json\n{formatted_json}\n```"))


wb.close()

Original:


```json
{
  "original_responsibility": {
    "0.responsibilities.0": "Conversational AI & NLP: Engineered the core dialogue system to generate dynamic questions,
evaluate responses, and maintain conversation flow.",
    "0.responsibilities.1": "Thought Generation & Processing: Designed pipelines for hierarchical idea expansion,
clustering, and ranking AI-generated thoughts.",
    "0.responsibilities.2": "Automated Evaluation & Adaptation: Developed an AI-based evaluator to assess user
responses, generate adaptive follow-ups, and track discussion depth.",
    "0.responsibilities.3": "State & Topic Management: Built a stateful tracking system to guide conversations, handle
topic transitions, and prevent redundancy.",
    "0.responsibilities.4": "Asynchronous AI Integration: Optimized API performance for OpenAI GPT-4 & Anthropic Claude
using AsyncIO, improving efficiency and scalability.",
    "1.responsibilities.0": "Provided strategic insights to a major global IT vendor, optimizing their service partner
ecosystem in Asia Pacific for improved local implementation outcomes.",
    "1.responsibilities.1": "Assisted a U.S.-based international services provider in its growth strategy by precisely
evaluating and scaling new engineering service opportunities in vital emerging markets.",
    "1.responsibilities.2": "Co-authored an industry-recognized report on M&A in the engineering services sector,
offering deep dives into deal sizes, capability gaps, and emerging opportunities, influencing strategic decisions in IT
and operational technology convergence.",
    "1.responsibilities.3": "Enhanced data quality and consistency by integrating thorough financial analysis,
standardizing methodologies, and conducting in-depth vendor engagements.",
    "1.responsibilities.4": "Achieved over 40% centralization of tasks to an offshore team in India, optimizing resource
allocation.",
    "1.responsibilities.5": "Developed Python tools to automate and accelerate internal processes, cutting report
preparation and data analysis time by over 40%.",
    "1.responsibilities.6": "Collaborated with the engineering services research team to pioneer the engineering
services tracker, authored influential publications on market forecasts, the impact of COVID-19 on services, and trends
in M&A within the engineering services industry.",
    "1.responsibilities.7": "Collaborated with analyst teams across the globe (US, Canada, Latin America, Europe, MEA,
APAC) to ensure data quality, meeting deadlines, sharing knowledge/best practices/methodology, and procure new tools.",
    "2.responsibilities.0": "Full P&L management including budgeting, HR, vendors, partnerships, research, and business
development: expanded the programs' bookings by more than 50%.",
    "2.responsibilities.1": "Expanded and managed a global research team of more than 15 (US, India, and Mexico).",
    "2.responsibilities.2": "Led an external software development team to build and implement new tools.",
    "2.responsibilities.3": "Championed new technology projects using ML, NLP, chatbot, ontology, web-scraping, API, UX
(User Experience). First in IDC to implement ML (machine learning). First to launch a chatbot. First to implement API
(application programming interface). First to build an ontology.",
    "2.responsibilities.4": "Led a team to develop automated tools in Python, reducing report preparation time by 40%.",
    "2.responsibilities.5": "Advised services firms on deal pursuit and sales orchestration strategies.",
    "2.responsibilities.6": "Advised software vendors on services partnership strategy.",
    "2.responsibilities.7": "Authored reports, blogs, presentations, & custom researches in go-to-market strategy, deal
signing analysis, renewal analysis, buyer studies, technology adoptions (cloud, AI, ML, digital, etc.), and industry
trend analysis.",
    "3.responsibilities.0": "Delivered quarterly webinars on outsourcing/managed services signing trends.",
    "3.responsibilities.1": "Authored 7 to 10 pursuit strategy reports and industry trend research documents.",
    "4.responsibilities.0": "Designed and architected a complex company database with 100K+ unique records and
integrating DnB API and internal databases, which reduced data duplication by 50%.",
    "4.responsibilities.1": "Oversaw two Appian implementations that increased team productivity by 20 to 30%.",
    "4.responsibilities.2": "Managed daily content operations, including leading a team of offshore and nearshore
content team, as well as other sales and research related activities.",
    "4.responsibilities.3": "Analyzed and Modeled 20 to 30 IT vendors' financials and over 1,500 services contracts.",
    "4.responsibilities.4": "Managed three major data integration projects critical to the launch of the new platform.",
    "4.responsibilities.5": "Promoted from Senior Research Analyst to Research Manager in February 2007.",
    "5.responsibilities.0": "Researched market dynamics in the Web Services Security market to support product
strategy."
  }
}
```

#### JSON format: edited by LLM

In [ ]:
file_path = r"C:\github\job_bot\input_output\human_review\resps_reqs_matching\reviewed_matchings\Accenture_Enterprise_AI_Value_Strategy_Senior_Manager_crosstab.xlsx"
sheet_name = "Crosstab"

wb, sheet = load_excel_sheet(file_path, sheet_name)
df = get_underlined_cells(sheet)
cleaned_df = clean_and_remove_rows(df)
final_df = rename_dataframe_columns(cleaned_df)

final_df.drop("original_responsibility", axis=1, inplace=True)
# print(final_df)

json_string = final_df.to_json()
json_data = json.loads(json_string)  # Convert string to dictionary

print("Edited by LLM:")
# Format JSON for readability
formatted_json = format_json_readable(json_data, wrap_width=120)

# Display with Markdown to prevent horizontal scrolling
display(Markdown(f"```json\n{formatted_json}\n```"))

wb.close()

Edited by LLM:


```json
{
  "edited_responsibility_1": {
    "0.responsibilities.0": "Engineered and executed AI-first strategies, focusing on the development and engineering of
conversational AI systems to enhance user engagement and generate commercialization opportunities.",
    "0.responsibilities.1": "Thought Generation & Processing: Developed advanced algorithms for structuring and
prioritizing AI-generated ideas, demonstrating expertise in environments akin to consulting, professional services, or
Big Four firms.",
    "0.responsibilities.2": "Developed an AI-based system to assess user interactions and adaptively generate follow-up
content, ensuring alignment with responsible AI practices as outlined in the strategic vision. This system also tracks
engagement levels to support an innovative operating model, fostering a culture of innovation and experimentation.",
    "0.responsibilities.3": "Built a stateful AI system to optimize conversation flows and strategically manage
dialogues, supporting the creation of strategies for AI-first products and exploring commercialization opportunities.",
    "0.responsibilities.4": "Asynchronous AI Integration: Led enhancements in API performance for leading AI models such
as OpenAI GPT-4 and Anthropic Claude, focusing on boosting efficiency and scalability. This strategic initiative was
crucial in reshaping the vision and opening pathways for data and AI-driven business transformation.",
    "1.responsibilities.0": "Optimized the service partner ecosystem for a leading global IT vendor in the Asia Pacific
region by providing strategic insights, including Data & AI, that enhanced client relationships and established
credibility as a trusted advisor, leading to improved implementation outcomes.",
    "1.responsibilities.1": "Assisted a U.S.-based international services provider by shaping strategic visions and
creating scalable opportunities, particularly in new data and AI-driven engineering services in key emerging markets.",
    "1.responsibilities.2": "Co-authored a seminal report on M&A in the engineering services sector, analyzing deal
dimensions and capability gaps. This work influenced strategic visions and fostered new opportunities, particularly by
driving convergence between IT and operational technology to reinvent businesses through data and AI.",
    "1.responsibilities.3": "Enhanced data quality and consistency through comprehensive financial analysis and the
standardization of methodologies, collaborating extensively with vendors to shape vision and unlock opportunities for
data & AI-led business reinvention.",
    "1.responsibilities.4": "Optimized resource allocation and enhanced team performance by centralizing over 40% of
tasks to an offshore team in India, effectively mentoring and developing high-performing teams.",
    "1.responsibilities.5": null,
    "1.responsibilities.6": "Led the engineering services research team in pioneering the engineering services tracker,
authored key publications on market forecasts, the impact of COVID-19 on the industry, and M&A trends, significantly
shaping strategic insights and creating opportunities for data & AI-led business reinvention.",
    "1.responsibilities.7": "Collaborated with analyst teams around the world, including the US, Canada, Latin America,
Europe, MEA, and APAC, in a global professional services environment. Enhanced data quality, ensured timely project
completions, and integrated new tools and best practices typical of Big Four and consulting firm settings.",
    "2.responsibilities.0": "Managed full P&L responsibilities encompassing budgeting, human resources, vendor
relations, and strategic partnerships, coupled with a focus on research and business development, which collectively led
to a substantial increase in program bookings.",
    "2.responsibilities.1": "Expanded and managed a multinational research team across the US, India, and Mexico,
focusing on spearheading the strategic development of AI-first products and their commercialization strategies.",
    "2.responsibilities.2": "Led and mentored high-performing external software development teams to innovate and
successfully implement new tools.",
    "2.responsibilities.3": "Championed AI-first product strategies, pioneering and commercializing cutting-edge
technologies including machine learning, natural language processing, chatbots, ontologies, and APIs. Spearheaded the
initial rollout of user-centric AI solutions that enhanced web interfaces and user experiences, opening significant
avenues for commercialization.",
    "2.responsibilities.4": "Led a team in developing Python-based automation tools, reducing report preparation time by
40%. Boasts extensive experience in business development, client relationship management, and marketing, with over five
years dedicated to these areas.",
    "2.responsibilities.5": "Advised service firms on integrating Responsible AI into their strategic vision and
roadmap, developed approaches for leveraging ecosystem partners in deal pursuits, and defined sales orchestration
processes to enhance a culture of innovation and experimentation.",
    "2.responsibilities.6": "Advised on strategic plans for AI-first software products, focusing on developing
partnership strategies and exploiting commercialization opportunities.",
    "2.responsibilities.7": "Authored detailed reports, blogs, presentations, and specialized research in go-to-market
strategies, deal signing, renewal processes, and buyer behavior, focusing on the in-depth analysis of technology
adoption trends such as cloud computing, artificial intelligence, machine learning, and digital transformations. This
expertise has played a crucial role in strengthening client relationships and establishing a reputation as a trusted
advisor, especially in guiding the integration of Data & AI into business processes and industry trends.",
    "3.responsibilities.0": "Delivered quarterly webinars on outsourcing and managed services, emphasizing strategic
planning and the utilization of partnerships to promote a culture of innovation and experimentation, in alignment with
Responsible AI and ecosystem collaboration trends.",
    "3.responsibilities.1": "Authored multiple strategy reports and conducted industry trend analyses, significantly
enhancing client relationships and establishing credibility as a trusted advisor on incorporating Data & AI into
business processes.",
    "4.responsibilities.0": "Designed and architected a sophisticated company database encompassing over 100,000 unique
records, integrating external APIs like Salesforce and internal systems, effectively reducing data duplication by 50%
and enhancing data integrity, demonstrating adeptness in CRM tools for efficient tracking and analysis of client
interactions.",
    "4.responsibilities.1": "Oversaw multiple Appian implementations at a Big Four firm, enhancing team productivity
significantly and demonstrating extensive consulting experience.",
    "4.responsibilities.2": "Managed daily content operations and led a diverse team, significantly enhancing
performance in sales and research, and demonstrating a strong ability to develop high-performing teams.",
    "4.responsibilities.3": "Analyzed and modeled the financials of numerous IT vendors, managed extensive portfolios of
service contracts, and leveraged this expertise to build strong client relationships, serving as a trusted advisor on
integrating Data & AI into business processes.",
    "4.responsibilities.4": "Led several major data integration projects, essential for the successful launch of a new
platform, and successfully mentored and developed high-performing teams.",
    "4.responsibilities.5": null,
    "5.responsibilities.0": "Explored market trends in the Web Services Security sector to bolster product strategy
through collaborative leadership and a growth-oriented approach."
  }
}
```

#### Final trim & edit by LLM

##### Trimmed by GPT

In [ ]:
from prompts.prompt_templates import TRIM_CONDENSE_FINAL_RESUME_WITH_JOB_DESC_PROMPT
from models.resume_job_description_io_models import JobFileMappings
from evaluation_optimization.create_mapping_file import load_mappings_model_from_json

# Set file path and url
file_path = accenture_file
url = (
    "https://www.accenture.com/us-en/careers/jobdetails?id=R00251798_en&src=LINKEDINJP"
)


# * Fetch edited resume in json format
sheet_name = "Crosstab"
wb, sheet = load_excel_sheet(file_path, sheet_name)

df = get_underlined_cells(sheet)
cleaned_df = clean_and_remove_rows(df)
final_df = rename_dataframe_columns(cleaned_df)
final_df.drop("original_responsibility", axis=1, inplace=True)

resps_json = final_df.to_json()


# * Fetch job_description
job_descriptions_file = JOB_DESCRIPTIONS_JSON_FILE
job_descriptions_dict = load_and_clean_json_file(job_descriptions_file)

job_content = job_descriptions_dict.get(url, {}).get("data", {}).get("content", {})
# display_json_pretty(job_content)


# * Insert JSON into template
prompt_template = TRIM_CONDENSE_FINAL_RESUME_WITH_JOB_DESC_PROMPT
final_prompt = prompt_template.format(
    resume_json=resps_json, job_description_json=job_content
)

print(final_prompt)

2025-03-20 23:42:43,257 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\preprocessing\jobpostings.json



You are an expert resume optimizer for senior-level tech and consulting roles.
Your task is to **trim, consolidate, and optimize** the provided JSON resume data while ensuring alignment with the job description provided.

### **Task Instructions**
1. **Trim & Condense:** Reduce the resume word count to approximately **500-600 words** while retaining essential details.
2. **Merge Related Content:** Combine overlapping or redundant bullets to enhance readability and conciseness.
3. **Enhance ATS Compliance:** Ensure the use of **strong action verbs**, **industry-specific keywords**, and **quantifiable achievements** (e.g., "% improvements," "increased revenue by X%," "cut processing time by Y%").
4. **Align with Job Description:** Ensure the resume highlights **key skills, experience, and qualifications** relevant to the job description.
5. **Ensure Consistency & Clarity:** Maintain a **structured, professional format**, ensuring smooth readability while preserving key impact points.
6.

In [ ]:
# * Output

file_path = r"C:\github\job_bot\input_output\human_review\resps_reqs_matching_trimmed\Accenture_Enterprise_AI_Value_Strategy_Senior_Manager_crosstab_trimmed.json"
display_json_pretty(file_path)

2025-03-20 23:43:34,427 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\human_review\resps_reqs_matching_trimmed\Accenture_Enterprise_AI_Value_Strategy_Senior_Manager_crosstab_trimmed.json


```json
{
  "edited_responsibility_1": {
    "0.responsibilities.0": "Engineered and executed AI-first strategies, focusing on conversational AI systems to boost
user engagement and drive commercialization.",
    "0.responsibilities.1": "Developed algorithms to structure and prioritize AI-generated ideas, reflecting strategic
thinking in consulting-style environments.",
    "0.responsibilities.2": "Built an adaptive AI system that assessed user interactions and generated follow-up
content, aligning with responsible AI principles and fostering a culture of innovation.",
    "0.responsibilities.3": "Designed stateful AI workflows to optimize conversation flow and identify commercialization
pathways for AI-first products.",
    "0.responsibilities.4": "Led asynchronous AI integration with GPT-4 and Claude, improving API performance and
scalability to accelerate data & AI transformation.",
    "1.responsibilities.0": "Advised a global IT vendor on optimizing its Asia Pacific service partner ecosystem,
establishing credibility as a trusted advisor and improving implementation outcomes.",
    "1.responsibilities.1": "Shaped strategic visions and created scalable Data & AI-driven engineering services for an
international services provider in emerging markets.",
    "1.responsibilities.2": "Co-authored a landmark M&A report in engineering services, identifying capability gaps and
catalyzing IT and OT convergence opportunities.",
    "1.responsibilities.3": "Standardized financial analysis methodologies to enhance data quality and unlock AI-led
reinvention opportunities for vendors.",
    "1.responsibilities.4": "Improved efficiency by centralizing over 40% of operations offshore and mentoring high-
performing global teams.",
    "1.responsibilities.6": "Led the engineering services tracker and authored key reports on market forecasts, COVID-19
impact, and M&A, influencing AI-driven business strategy.",
    "1.responsibilities.7": "Collaborated across analyst teams in six global regions to enhance data quality and ensure
on-time delivery, integrating best practices in a Big Four-style setting.",
    "2.responsibilities.0": "Held full P&L responsibility including budgeting, HR, vendor partnerships, and research,
contributing to a 50%+ increase in program bookings.",
    "2.responsibilities.1": "Managed global research teams across the US, India, and Mexico, spearheading the
development and commercialization of AI-first products.",
    "2.responsibilities.2": "Led external software teams in building new tools, delivering innovation through agile
implementation.",
    "2.responsibilities.3": "Pioneered AI-first product strategies including ML, NLP, chatbots, and APIs; launched user-
centric AI tools that enhanced UX and opened commercialization channels.",
    "2.responsibilities.4": "Developed Python-based automation tools, cutting report prep time by 40%; brought 5+ years
of business development and client engagement expertise.",
    "2.responsibilities.5": "Guided firms on integrating Responsible AI into strategy, leveraging ecosystem partnerships
and defining innovative sales processes.",
    "2.responsibilities.6": "Advised on AI-first product roadmaps, partnership strategies, and commercialization
initiatives.",
    "2.responsibilities.7": "Produced high-impact reports, presentations, and blogs on GTM strategies, renewals, and
tech adoption (cloud, AI/ML), strengthening client relationships.",
    "3.responsibilities.0": "Led quarterly webinars on outsourcing and managed services, promoting innovation through
Responsible AI and ecosystem strategies.",
    "3.responsibilities.1": "Authored strategy reports and trend analyses, deepening client trust and enabling Data &
AI-driven business transformation.",
    "4.responsibilities.0": "Designed a company database of 100K+ records with Salesforce integration, cutting data
duplication by 50% and improving CRM efficiency.",
    "4.responsibilities.1": "Oversaw Appian implementations at a Big Four firm, enhancing productivity and showcasing
deep consulting acumen.",
    "4.responsibilities.2": "Directed a diverse team to elevate sales and research performance, demonstrating leadership
in high-performing team development.",
    "4.responsibilities.3": "Analyzed IT vendor financials and managed service contract portfolios to advise clients on
Data & AI integration.",
    "4.responsibilities.4": "Led major data integration projects enabling platform launches, while mentoring top-
performing global teams.",
    "5.responsibilities.0": "Researched Web Services Security market trends to inform product strategy through
collaborative leadership and growth-oriented insights."
  }
}
```

In [ ]:
file_path = r"C:\github\job_bot\input_output\human_review\resps_reqs_matching_trimmed\Accenture_Enterprise_AI_Value_Strategy_Senior_Manager_crosstab_trimmed.json"
data = load_and_clean_json_file(file_path)

edited_resps = data["edited_responsibility_1"]

formatted_text = "\n\n".join(f"{key}\n{value}" for key, value in edited_resps.items())

print(formatted_text)

2025-03-20 23:45:05,508 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\human_review\resps_reqs_matching_trimmed\Accenture_Enterprise_AI_Value_Strategy_Senior_Manager_crosstab_trimmed.json


0.responsibilities.0
Engineered and executed AI-first strategies, focusing on conversational AI systems to boost user engagement and drive commercialization.

0.responsibilities.1
Developed algorithms to structure and prioritize AI-generated ideas, reflecting strategic thinking in consulting-style environments.

0.responsibilities.2
Built an adaptive AI system that assessed user interactions and generated follow-up content, aligning with responsible AI principles and fostering a culture of innovation.

0.responsibilities.3
Designed stateful AI workflows to optimize conversation flow and identify commercialization pathways for AI-first products.

0.responsibilities.4
Led asynchronous AI integration with GPT-4 and Claude, improving API performance and scalability to accelerate data & AI transformation.

1.responsibilities.0
Advised a global IT vendor on optimizing its Asia Pacific service partner ecosystem, establishing credibility as a trusted advisor and improving implementation outcom

##### Trimmed by Grok

In [ ]:
file_path = r"C:\github\job_bot\data\Accenture_Enterprise_AI_Value_Strategy_Senior_Manager_grok_final_edit.json"

display_json_pretty(file_path)

2025-03-20 13:52:03,720 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\data\Accenture_Enterprise_AI_Value_Strategy_Senior_Manager_grok_final_edit.json


```json
{
  "revised_responsibility_1": {
    "0.responsibilities.0": "Engineered conversational AI systems to boost engagement and drive commercialization
opportunities.",
    "0.responsibilities.1": "Developed advanced algorithms to structure and prioritize AI-generated ideas for strategic
applications.",
    "0.responsibilities.2": "Built AI system to evaluate user interactions, adapt content, and measure engagement
depth.",
    "0.responsibilities.3": "Created stateful AI framework to optimize conversation flows and guide strategic
dialogues.",
    "0.responsibilities.4": "Enhanced API performance for OpenAI GPT-4 and Anthropic Claude with AsyncIO for
scalability.",
    "1.responsibilities.0": "Optimized IT vendor’s Asia Pacific ecosystem with data and AI-driven strategic insights.",
    "1.responsibilities.1": "Shaped scalable data and AI engineering service strategies for a U.S. provider in emerging
markets.",
    "1.responsibilities.2": "Co-authored M&A report on engineering services, driving IT-OT convergence strategies.",
    "1.responsibilities.3": "Improved data quality with financial analysis and vendor collaboration for AI-led
outcomes.",
    "1.responsibilities.4": "Centralized 40% of tasks to offshore India team, enhancing resource efficiency.",
    "1.responsibilities.5": null,
    "1.responsibilities.6": "Led engineering services tracker, authored influential market forecasts and M&A trend
analyses.",
    "1.responsibilities.7": "Collaborated globally to ensure data quality, meet deadlines, and deploy innovative
tools.",
    "2.responsibilities.0": "Managed P&L across budgeting and partnerships, growing bookings by over 50%.",
    "2.responsibilities.1": "Expanded and led global research team in US, India, and Mexico for AI-first initiatives.",
    "2.responsibilities.2": "Directed external software team to develop and deploy innovative tools.",
    "2.responsibilities.3": "Pioneered ML, NLP, and chatbot technologies, launching AI-first solutions.",
    "2.responsibilities.4": "Led Python tool development, cutting report preparation time by 40%.",
    "2.responsibilities.5": "Advised firms on Responsible AI integration and sales orchestration strategies.",
    "2.responsibilities.6": "Guided software vendors on AI-first partnership and commercialization strategies.",
    "2.responsibilities.7": "Authored go-to-market reports on AI, cloud, and digital trends for strategic impact.",
    "3.responsibilities.0": "Delivered webinars on outsourcing trends, emphasizing innovation and partnerships.",
    "3.responsibilities.1": "Authored strategy reports and trend analyses to shape data and AI insights.",
    "4.responsibilities.0": "Architected database with 100K+ records, cutting duplication by 50% via API integration.",
    "4.responsibilities.1": "Oversaw Appian implementations, increasing team productivity by 20-30%.",
    "4.responsibilities.2": "Managed content operations, leading offshore teams for sales and research success.",
    "4.responsibilities.3": "Analyzed financials of IT vendors and 1,500+ contracts for strategic insights.",
    "4.responsibilities.4": "Led critical data integration projects for new platform deployment.",
    "4.responsibilities.5": null,
    "5.responsibilities.0": "Researched Web Services Security trends to inform product strategy."
  }
}
```

In [ ]:
file_path = r"C:\github\job_bot\data\Accenture_Enterprise_AI_Value_Strategy_Senior_Manager_grok_final_edit.json"
data = load_and_clean_json_file(file_path)

edited_resps = data["revised_responsibility_1"]

formatted_text = "\n\n".join(f"{key}\n{value}" for key, value in edited_resps.items())

print(formatted_text)

2025-03-20 23:48:55,625 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\data\Accenture_Enterprise_AI_Value_Strategy_Senior_Manager_grok_final_edit.json


0.responsibilities.0
Engineered conversational AI systems to boost engagement and drive commercialization opportunities.

0.responsibilities.1
Developed advanced algorithms to structure and prioritize AI-generated ideas for strategic applications.

0.responsibilities.2
Built AI system to evaluate user interactions, adapt content, and measure engagement depth.

0.responsibilities.3
Created stateful AI framework to optimize conversation flows and guide strategic dialogues.

0.responsibilities.4
Enhanced API performance for OpenAI GPT-4 and Anthropic Claude with AsyncIO for scalability.

1.responsibilities.0
Optimized IT vendor’s Asia Pacific ecosystem with data and AI-driven strategic insights.

1.responsibilities.1
Shaped scalable data and AI engineering service strategies for a U.S. provider in emerging markets.

1.responsibilities.2
Co-authored M&A report on engineering services, driving IT-OT convergence strategies.

1.responsibilities.3
Improved data quality with financial analysis 

## Airtable Job Posting

In [9]:
import pandas as pd

# pd.set_option("display.max_columns", None)  # show all columns
# pd.set_option("display.max_rows", None)  # show all rows
# pd.set_option("display.width", 500)  # allow wide output
# pd.set_option("display.max_colwidth", 500)  # show full cell content
# pd.reset_option("all")

### Get file name and url

In [8]:
import xlwings as xw
import pandas as pd
from IPython.display import display
from utils.get_file_names import get_file_names
from evaluation_optimization.create_mapping_file import load_mappings_model_from_json
from project_config import (
    ITERATE_1_OPENAI_DIR,
    mapping_file_name,
    REVIEWED_MATCHINGS_DIR,
)

# Get matching files
file_list = get_file_names(REVIEWED_MATCHINGS_DIR, full_path=True)

# Match company file (case-insensitive)
company = "Airtable"
company_file = next(
    (file for file in file_list if company.lower() in str(file).lower()), None
)

if not company_file:
    raise FileNotFoundError(f"No reviewed matchings file found for {company}")

# Load mapping file model
mapping_file = ITERATE_1_OPENAI_DIR / mapping_file_name
mapping_file_model = load_mappings_model_from_json(mapping_file)

# print(mapping_file_model)


# Get company URL(s) from mapping model
company_url = next(
    str(key) for key in mapping_file_model.root if company.lower() in str(key).lower()
)

if not company_url:
    print(f"⚠️ No URLs found for company '{company}' in mapping file.")

# Output
print(f"\n✅ {company} file:\n{company_file}")
print(f"\n🔗 {company} URL(s):{company_url}")

2025-03-26 09:42:12,489 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\evaluation_optimization\evaluation_optimization_by_openai\iteration_1\url_to_file_mapping.json
2025-03-26 09:42:12,490 - evaluation_optimization.create_mapping_file - INFO - Loaded and validated mapping file from C:\github\job_bot\input_output\evaluation_optimization\evaluation_optimization_by_openai\iteration_1\url_to_file_mapping.json



✅ Airtable file:
C:\github\job_bot\input_output\human_review\resps_reqs_matchings_reviewed\Airtable_Product_Manager__AI_crosstab.xlsx

🔗 Airtable URL(s):https://job-boards.greenhouse.io/airtable/jobs/7603873002?gh_src=aef790d02us


In [11]:
# Set file and url vars

airtable_file = company_file
airtable_url = company_url

### Dataframe format, prioritized edited responsibilities manually

In [12]:
import xlwings as xw
import pandas as pd  # ✅ Required for pd.set_option and display
from IPython.display import display
from utils.get_file_names import get_file_names
from project_config import REVIEWED_MATCHINGS_DIR


# ✅ Get matching files
file_list = get_file_names(REVIEWED_MATCHINGS_DIR, full_path=True)


cross_tab_file = company_file

sheet_name = "Crosstab"

try:
    if not cross_tab_file:
        raise FileNotFoundError(f"No file found for company: {company}")

    df_all_data = pd.read_excel(cross_tab_file)

    wb, sheet = load_excel_sheet(file_path=cross_tab_file, sheet_name=sheet_name)
    df, coordinates = get_underlined_cells(sheet=sheet, return_coords=True)

except Exception as e:
    print(f"Error loading job posting: {e}")
    df = None


# df is your DataFrame
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

# ✅ Continue only if DataFrame loaded successfully
if df is not None:
    cleaned_df = clean_and_remove_rows(df)
    final_df = rename_dataframe_columns(cleaned_df)

    print(company)
    print(cross_tab_file)
    # pd.set_option("display.max_columns", None)
    display(final_df)

    wb.close()

pd.set_option("display.max_colwidth", 90)  # or a large number like 200

# pyright: ignore[reportInvalidTypeForm]
summary_df = summarize_underlined_column_frequencies(
    df_all_data=df_all_data,
    underlined_coords=coordinates,
)
print(f"Requirements Balance: \n{summary_df}")
pd.reset_option("all")

Airtable
C:\github\job_bot\input_output\human_review\resps_reqs_matchings_reviewed\Airtable_Product_Manager__AI_crosstab.xlsx


,original_responsibility,edited_responsibility_1
responsibility_key,,
0.responsibilities.0,Conversational AI & NLP: Engineered the core d...,Conversational AI & NLP: Engineered innovative...
0.responsibilities.1,Thought Generation & Processing: Designed pipe...,Analytical and Problem-Solving Excellence: Des...
0.responsibilities.2,Automated Evaluation & Adaptation: Developed a...,Developed an AI-based evaluator to assess user...
0.responsibilities.3,State & Topic Management: Built a stateful tra...,"Built and owned a stateful tracking system, de..."
0.responsibilities.4,Asynchronous AI Integration: Optimized API per...,Asynchronous AI Integration: Optimized API per...
1.responsibilities.0,Provided strategic insights to a major global ...,Collaborated directly with enterprise clients ...
1.responsibilities.1,Assisted a U.S.-based international services p...,Effectively articulated and expanded the strat...
1.responsibilities.2,Co-authored an industry-recognized report on M...,Co-authored a respected report on mergers and ...
1.responsibilities.3,Enhanced data quality and consistency by integ...,Enhanced analytical and problem-solving skills...


Requirements Balance: 
                                                                                 Column Name  Underline Count  Frequency (%)
1  Experience working directly with enterprise customers and partnering closely with sale...                5           17.9
7  Excellent communication and collaboration skills with ability to work effectively acro...                5           17.9
0  Excellent analytical and problem solving skills with demonstrated ability to make data...                4           14.3
4  Strong communication and presentation skills with ability to articulate compelling pro...                4           14.3
2                              Familiarity with Airtable low code productivity tools or SaaS                3           10.7
3  Proven track record of shaping and delivering large ambiguous 0 to 1 features or products                3           10.7
6                 Track record of taking ownership showing initiative and delivering results          

C:\Users\xzhan\AppData\Local\Temp\ipykernel_35600\2598938058.py:54: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option("all")
C:\Users\xzhan\AppData\Local\Temp\ipykernel_35600\2598938058.py:54: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option("all")


### JSON format: Original, prioritized edited responsibilities manually

In [13]:
file_path = airtable_file
sheet_name = "Crosstab"

if file_path is not None:
    wb, sheet = load_excel_sheet(file_path, sheet_name)
    df = get_underlined_cells(sheet)
    cleaned_df = clean_and_remove_rows(df)
    final_df = rename_dataframe_columns(cleaned_df)
else:
    raise FileExistsError

final_df.drop("edited_responsibility_1", axis=1, inplace=True)

json_string = final_df.to_json()
json_data = json.loads(json_string)  # Convert string to dictionary

print("JSON: original")
display_json_pretty(json_data)

wb.close()

JSON: original


```json
{
  "original_responsibility": {
    "0.responsibilities.0": "Conversational AI & NLP: Engineered the core dialogue system to generate dynamic questions,
evaluate responses, and maintain conversation flow.",
    "0.responsibilities.1": "Thought Generation & Processing: Designed pipelines for hierarchical idea expansion,
clustering, and ranking AI-generated thoughts.",
    "0.responsibilities.2": "Automated Evaluation & Adaptation: Developed an AI-based evaluator to assess user
responses, generate adaptive follow-ups, and track discussion depth.",
    "0.responsibilities.3": "State & Topic Management: Built a stateful tracking system to guide conversations, handle
topic transitions, and prevent redundancy.",
    "0.responsibilities.4": "Asynchronous AI Integration: Optimized API performance for OpenAI GPT-4 & Anthropic Claude
using AsyncIO, improving efficiency and scalability.",
    "1.responsibilities.0": "Provided strategic insights to a major global IT vendor, optimizing their service partner
ecosystem in Asia Pacific for improved local implementation outcomes.",
    "1.responsibilities.1": "Assisted a U.S.-based international services provider in its growth strategy by precisely
evaluating and scaling new engineering service opportunities in vital emerging markets.",
    "1.responsibilities.2": "Co-authored an industry-recognized report on M&A in the engineering services sector,
offering deep dives into deal sizes, capability gaps, and emerging opportunities, influencing strategic decisions in IT
and operational technology convergence.",
    "1.responsibilities.3": "Enhanced data quality and consistency by integrating thorough financial analysis,
standardizing methodologies, and conducting in-depth vendor engagements.",
    "1.responsibilities.4": "Achieved over 40% centralization of tasks to an offshore team in India, optimizing resource
allocation.",
    "1.responsibilities.5": "Developed Python tools to automate and accelerate internal processes, cutting report
preparation and data analysis time by over 40%.",
    "1.responsibilities.6": "Collaborated with the engineering services research team to pioneer the engineering
services tracker, authored influential publications on market forecasts, the impact of COVID-19 on services, and trends
in M&A within the engineering services industry.",
    "1.responsibilities.7": "Collaborated with analyst teams across the globe (US, Canada, Latin America, Europe, MEA,
APAC) to ensure data quality, meeting deadlines, sharing knowledge/best practices/methodology, and procure new tools.",
    "2.responsibilities.0": "Full P&L management including budgeting, HR, vendors, partnerships, research, and business
development: expanded the programs' bookings by more than 50%.",
    "2.responsibilities.1": "Expanded and managed a global research team of more than 15 (US, India, and Mexico).",
    "2.responsibilities.2": "Led an external software development team to build and implement new tools.",
    "2.responsibilities.3": "Championed new technology projects using ML, NLP, chatbot, ontology, web-scraping, API, UX
(User Experience). First in IDC to implement ML (machine learning). First to launch a chatbot. First to implement API
(application programming interface). First to build an ontology.",
    "2.responsibilities.4": "Led a team to develop automated tools in Python, reducing report preparation time by 40%.",
    "2.responsibilities.5": "Advised services firms on deal pursuit and sales orchestration strategies.",
    "2.responsibilities.6": "Advised software vendors on services partnership strategy.",
    "2.responsibilities.7": "Authored reports, blogs, presentations, & custom researches in go-to-market strategy, deal
signing analysis, renewal analysis, buyer studies, technology adoptions (cloud, AI, ML, digital, etc.), and industry
trend analysis.",
    "3.responsibilities.0": "Delivered quarterly webinars on outsourcing/managed services signing trends.",
    "3.responsibilities.1": "Authored 7 to 10 pursuit strategy reports and industry trend research documents.",
    "4.responsibilities.0": "Designed and architected a complex company database with 100K+ unique records and
integrating DnB API and internal databases, which reduced data duplication by 50%.",
    "4.responsibilities.1": "Oversaw two Appian implementations that increased team productivity by 20 to 30%.",
    "4.responsibilities.2": "Managed daily content operations, including leading a team of offshore and nearshore
content team, as well as other sales and research related activities.",
    "4.responsibilities.3": "Analyzed and Modeled 20 to 30 IT vendors' financials and over 1,500 services contracts.",
    "4.responsibilities.4": "Managed three major data integration projects critical to the launch of the new platform.",
    "4.responsibilities.5": "Promoted from Senior Research Analyst to Research Manager in February 2007.",
    "5.responsibilities.0": "Researched market dynamics in the Web Services Security market to support product
strategy."
  }
}
```

### JSON format: edited by LLM, prioritized edited responsibilities manually

In [18]:
file_path = airtable_file
sheet_name = "Crosstab"

wb, sheet = load_excel_sheet(file_path, sheet_name)

df = get_underlined_cells(sheet)
cleaned_df = clean_and_remove_rows(df)

final_df = rename_dataframe_columns(cleaned_df)
final_df.drop("original_responsibility", axis=1, inplace=True)


# print(final_df)
json_string = final_df.to_json()
# json_data = json.loads(json_string)  # Convert string to dictionary

print(company)
print("Edited by LLM:")

# # Format JSON for readability
# formatted_json = format_json_readable(json_data, wrap_width=120)

# # Display with Markdown to prevent horizontal scrolling
# display(Markdown(f"```json\n{formatted_json}\n```"))


display_json_pretty(json_string)
wb.close()

Airtable
Edited by LLM:


```json
{
  "edited_responsibility_1": {
    "0.responsibilities.0": "Conversational AI & NLP: Engineered innovative dialogue systems from concept to completion,
adept at generating dynamic questions, evaluating responses, and managing conversation flow, demonstrating a consistent
ability to shape and deliver large, ambiguous 0 to 1 features or products.",
    "0.responsibilities.1": "Analytical and Problem-Solving Excellence: Designed frameworks to organize and prioritize
AI-generated concepts, enabling rapid, data-driven, and customer-informed decision-making amidst imperfect
information.",
    "0.responsibilities.2": "Developed an AI-based evaluator to assess user responses and generate adaptive follow-ups,
enhancing customer engagement significantly. Collaborated closely with enterprise customers and partnered with sales and
marketing teams to analyze discussion depth and support strategic objectives.",
    "0.responsibilities.3": "Built and owned a stateful tracking system, demonstrating initiative by efficiently
managing and transitioning conversation topics, which enhanced communication and achieved measurable reductions in
redundancies.",
    "0.responsibilities.4": "Asynchronous AI Integration: Optimized API performance for OpenAI GPT-4 & Anthropic Claude
using AsyncIO, improving efficiency and scalability. Articulated a compelling product vision and strategy to both
technical and non-technical audiences, demonstrating strong communication and presentation skills.",
    "1.responsibilities.0": "Collaborated directly with enterprise clients and partnered with sales and marketing teams
to enhance strategic service implementations across the Asia Pacific region for a major global IT vendor.",
    "1.responsibilities.1": "Effectively articulated and expanded the strategic vision of engineering services to both
technical and non-technical audiences in key emerging markets for a U.S.-based international services provider,
demonstrating strong communication and presentation skills.",
    "1.responsibilities.2": "Co-authored a respected report on mergers and acquisitions within the engineering services
sector, delivering detailed analyses of deal dimensions, capability discrepancies, and new prospects. This effort
influenced strategic decisions by integrating IT and operational technologies, demonstrating strong communication skills
and effectively presenting to both technical and non-technical stakeholders.",
    "1.responsibilities.3": "Enhanced analytical and problem-solving skills by integrating thorough financial analysis,
standardizing methodologies, and conducting in-depth vendor engagements, thereby enabling swift, well-informed decision-
making even in the face of imperfect information.",
    "1.responsibilities.4": "Achieved over 40% centralization of tasks to an offshore team in India, enhancing direct
interactions with enterprise customers and strengthening partnerships with sales and marketing teams through optimized
resource allocation.",
    "1.responsibilities.5": null,
    "1.responsibilities.6": "Collaborated with various departments to pioneer the engineering services tracker through
effective teamwork, notably with the engineering services research team. Authored influential publications on market
forecasts, the impact of COVID-19 on services, and trends in mergers and acquisitions within the engineering services
sector.",
    "1.responsibilities.7": "Took ownership and showed initiative by collaborating effectively with global analyst teams
in the US, Canada, Latin America, Europe, MEA, and APAC. Enhanced data quality, met project deadlines, and implemented
innovative tools, ensuring the delivery of outstanding results.",
    "2.responsibilities.0": "Managed full P&L, encompassing budget oversight, HR, vendor relations, and strategic
partnerships, with a strong emphasis on business development. Successfully launched and scaled pioneering features or
products from inception to market leadership, dramatically expanding program bookings.",
    "2.responsibilities.1": "Expanded and managed a multinational research team, enhancing analytical capabilities and
decision-making processes across the US, India, and Mexico. Demonstrated exceptional analytical and problem-solving
skills by driving rapid, data-informed, and customer-focused decisions in scenarios with incomplete information.",
    "2.responsibilities.2": "Led a software development team in transforming novel ideas into fully realized and
launched software products, successfully shaping and delivering large, innovative features from inception to market.",
    "2.responsibilities.3": "Pioneered the introduction of machine learning, natural language processing, chatbots,
ontologies, and web-scraping technologies, demonstrating strong communication and presentation skills. Led the adoption
and integration of APIs and enhanced user experience (UX), articulating these advancements and the compelling product
vision and strategy effectively to both technical and non-technical stakeholders.",
    "2.responsibilities.4": "Led a team in developing automated tools using Python, integrating with various SaaS
platforms similar to Airtable to enhance efficiency and slash report generation time.",
    "2.responsibilities.5": "Advised service firms on enhancing deal pursuits and coordinating sales efforts
effectively, demonstrating excellent communication and collaboration skills across various teams and departments.",
    "2.responsibilities.6": "Specialized in advising software vendors on services partnership strategy, with extensive
experience spanning over six years as a product manager.",
    "2.responsibilities.7": "Authored various reports, blogs, and presentations, and conducted custom research on
pivotal topics including go-to-market strategies and technology adoptions such as cloud, AI, and ML. Collaborated
effectively with multiple teams to analyze industry trends and cross-departmental interactions, demonstrating excellent
communication and strong collaboration skills.",
    "3.responsibilities.0": "Effectively delivered quarterly webinars on outsourcing and managed services, demonstrating
excellent communication and collaboration by working closely with various teams and departments.",
    "3.responsibilities.1": "Authored numerous pursuit strategy reports and conducted industry trend analyses,
consistently taking ownership and delivering impactful results.",
    "4.responsibilities.0": "Led a cross-departmental team in designing and architecting a complex company database
containing over 100,000 unique records, integrating the DnB API and internal databases, achieving a 50% reduction in
data duplication. This initiative demanded superior communication and collaboration skills for effective coordination
among diverse teams.",
    "4.responsibilities.1": "Oversaw multiple Appian implementations, enhancing team productivity and strengthening
collaborations with enterprise customers, alongside internal sales and marketing teams.",
    "4.responsibilities.2": "Managed daily content operations and worked directly with enterprise customers, while
partnering closely with sales and marketing teams. Led a diverse team of offshore and nearshore content specialists
focused on research and sales-related activities.",
    "4.responsibilities.3": "Analyzed and modeled financial data for a range of IT vendors and conducted thorough
reviews of over 1,500 service contracts, demonstrating excellent analytical and problem-solving skills. This approach
enabled quick, data-driven, and customer-informed decisions even with imperfect information.",
    "4.responsibilities.4": "Managed multiple key data integration initiatives, including the successful deployment of a
new SaaS platform, demonstrating proficiency with low code productivity tools such as Airtable.",
    "4.responsibilities.5": null,
    "5.responsibilities.0": "Researched market trends in SaaS-based Web Services, focusing on low code productivity
tools such as Airtable, to refine product strategies."
  }
}
```

### Final trim & edit by GPT, prioritized edited responsibilities manually

#### Create prompt

In [19]:
from prompts.prompt_templates import TRIM_CONDENSE_FINAL_RESUME_WITH_JOB_DESC_PROMPT
from models.resume_job_description_io_models import JobFileMappings
from evaluation_optimization.create_mapping_file import load_mappings_model_from_json

# Set file path and url
file_path = airtable_file
url = airtable_url


# * Fetch edited resume in json format
sheet_name = "Crosstab"
wb, sheet = load_excel_sheet(file_path, sheet_name)

df = get_underlined_cells(sheet)
cleaned_df = clean_and_remove_rows(df)
final_df = rename_dataframe_columns(cleaned_df)
final_df.drop("original_responsibility", axis=1, inplace=True)

resps_json = final_df.to_json()


# * Fetch job_description
job_descriptions_file = JOB_DESCRIPTIONS_JSON_FILE
job_descriptions_dict = load_and_clean_json_file(job_descriptions_file)
job_description = job_descriptions_dict[url]["data"]["content"]


# * Insert JSON into template
prompt_template = TRIM_CONDENSE_FINAL_RESUME_WITH_JOB_DESC_PROMPT
final_prompt = prompt_template.format(
    resume_json=resps_json, job_description_json=job_description
)

print(final_prompt)

2025-03-26 09:50:50,463 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\preprocessing\jobpostings.json



You are an expert resume optimizer for senior-level tech and consulting roles.
Your task is to **trim, consolidate, and optimize** the provided JSON resume data while ensuring alignment with the job description provided.

### **Task Instructions**
1. **Trim & Condense:** Reduce the resume word count to approximately **500-600 words** while retaining essential details.
2. **Merge Related Content:** Combine overlapping or redundant bullets to enhance readability and conciseness.
3. **Enhance ATS Compliance:** Ensure the use of **strong action verbs**, **industry-specific keywords**, and **quantifiable achievements** (e.g., "% improvements," "increased revenue by X%," "cut processing time by Y%").
4. **Align with Job Description:** Ensure the resume highlights **key skills, experience, and qualifications** relevant to the job description.
5. **Ensure Consistency & Clarity:** Maintain a **structured, professional format**, ensuring smooth readability while preserving key impact points.
6.

In [23]:
from pathlib import Path
from project_config import TRIMMED_MATCHINGS_DIR

file_path = Path(airtable_file)
trimmed_file_name = f"{file_path.stem}_trimmed.json"

trimmed_file = TRIMMED_MATCHINGS_DIR / trimmed_file_name
print(trimmed_file_name)

Airtable_Product_Manager__AI_crosstab_trimmed.json


#### LLM response output - by Grok AI

In [25]:
file_path = trimmed_file
data = load_and_clean_json_file(file_path)

word_count = 0


# Iterate through the values of the JSON object
def count_words(data):
    word_count = 0
    for v in data.values():
        if isinstance(v, dict):
            word_count += count_words(v)
        elif isinstance(v, str):
            word_count += len(v.split())
    return word_count


word_count = count_words(data)


print("Trimmed and Condensed by GPT:")
print(f"Word count: {word_count}")
display_json_pretty(file_path)

2025-03-26 09:55:23,542 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\human_review\resps_reqs_matchings_trimmed\Airtable_Product_Manager__AI_crosstab_trimmed.json
2025-03-26 09:55:23,545 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\human_review\resps_reqs_matchings_trimmed\Airtable_Product_Manager__AI_crosstab_trimmed.json


Trimmed and Condensed by GPT:
Word count: 483


```json
{
  "edited_responsibility_1": {
    "0.responsibilities.0": "Engineered conversational AI and NLP systems, delivering innovative dialogue solutions that
enhanced user engagement by 30% and streamlined conversation flow for enterprise clients.",
    "0.responsibilities.1": "Designed analytical frameworks to prioritize AI-generated concepts, enabling rapid, data-
driven decisions and reducing decision-making time by 25%.",
    "0.responsibilities.2": "Developed an AI evaluator to assess responses and generate adaptive follow-ups, increasing
customer interaction depth by 40% while collaborating with sales and marketing teams.",
    "0.responsibilities.3": "Built a stateful tracking system to manage conversation transitions, cutting redundancies
by 20% and improving communication efficiency.",
    "0.responsibilities.4": "Optimized API performance for GPT-4 and Claude using AsyncIO, boosting scalability by 35%,
and articulated product vision to diverse stakeholders.",
    "1.responsibilities.0": "Partnered with enterprise clients and sales teams to implement strategic AI services across
Asia Pacific, enhancing adoption by 45%.",
    "1.responsibilities.1": "Expanded engineering services vision in emerging markets, presenting strategies to
technical and non-technical audiences, driving 15% market growth.",
    "1.responsibilities.2": "Co-authored a mergers and acquisitions report, integrating IT and operational insights to
influence strategic decisions for a global IT vendor.",
    "1.responsibilities.3": "Standardized financial analysis methodologies, enabling swift, informed decisions and
improving vendor engagement efficiency by 30%.",
    "1.responsibilities.4": "Centralized 40% of tasks to an offshore team in India, optimizing resource allocation and
strengthening client partnerships.",
    "1.responsibilities.5": null,
    "1.responsibilities.6": "Pioneered an engineering services tracker, authoring market trend reports on AI and M&A,
boosting forecast accuracy by 20%.",
    "1.responsibilities.7": "Led global analyst teams across six regions, enhancing data quality and delivering
innovative tools on time.",
    "2.responsibilities.0": "Managed full P&L and strategic partnerships, scaling pioneering features from inception to
market leadership, growing bookings by 50%.",
    "2.responsibilities.1": "Expanded a multinational research team, driving data-informed decisions and cutting
analysis time by 25% across three countries.",
    "2.responsibilities.2": "Led software development team to launch innovative products, delivering large-scale
features from concept to market.",
    "2.responsibilities.3": "Introduced ML, NLP, and chatbot technologies, enhancing UX and API integration while
presenting strategies to diverse audiences.",
    "2.responsibilities.4": "Developed Python-based automation tools integrating with SaaS platforms like Airtable,
slashing report generation time by 40%.",
    "2.responsibilities.5": "Advised service firms on sales coordination, improving deal pursuit success rates by 20%
through cross-team collaboration.",
    "2.responsibilities.6": "Specialized in partnership strategies for software vendors, leveraging six years as a
product manager to drive AI adoption.",
    "2.responsibilities.7": "Authored reports on cloud, AI, and ML trends, collaborating with teams to analyze industry
shifts and inform GTM strategies.",
    "3.responsibilities.0": "Delivered quarterly webinars on outsourcing and managed services, boosting engagement
through cross-departmental collaboration.",
    "3.responsibilities.1": "Authored pursuit strategy reports and trend analyses, consistently delivering impactful
results with 90% stakeholder approval.",
    "4.responsibilities.0": "Led a team to design a database with 100,000+ records, integrating DnB API and reducing
data duplication by 50%.",
    "4.responsibilities.1": "Oversaw Appian implementations, enhancing team productivity by 30% and strengthening
enterprise client ties.",
    "4.responsibilities.2": "Managed content operations and offshore teams, partnering with sales and marketing to
support research and sales goals.",
    "4.responsibilities.3": "Modeled financial data for IT vendors, reviewing 1,500+ contracts to enable quick,
customer-focused decisions.",
    "4.responsibilities.4": "Deployed a SaaS platform with Airtable integration, streamlining data initiatives and
improving workflow efficiency by 25%.",
    "4.responsibilities.5": null,
    "5.responsibilities.0": "Researched SaaS trends in low-code tools like Airtable, refining product strategies and
boosting adoption by 15%."
  }
}
```

In [26]:
# In text copy/paste friendly format

file_path = trimmed_file
data = load_and_clean_json_file(file_path)

edited_resps = data["edited_responsibility_1"]

formatted_text = "\n\n".join(f"{key}\n{value}" for key, value in edited_resps.items())

print(formatted_text)

2025-03-26 09:55:28,812 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\human_review\resps_reqs_matchings_trimmed\Airtable_Product_Manager__AI_crosstab_trimmed.json


0.responsibilities.0
Engineered conversational AI and NLP systems, delivering innovative dialogue solutions that enhanced user engagement by 30% and streamlined conversation flow for enterprise clients.

0.responsibilities.1
Designed analytical frameworks to prioritize AI-generated concepts, enabling rapid, data-driven decisions and reducing decision-making time by 25%.

0.responsibilities.2
Developed an AI evaluator to assess responses and generate adaptive follow-ups, increasing customer interaction depth by 40% while collaborating with sales and marketing teams.

0.responsibilities.3
Built a stateful tracking system to manage conversation transitions, cutting redundancies by 20% and improving communication efficiency.

0.responsibilities.4
Optimized API performance for GPT-4 and Claude using AsyncIO, boosting scalability by 35%, and articulated product vision to diverse stakeholders.

1.responsibilities.0
Partnered with enterprise clients and sales teams to implement strategic AI se

In [ ]:
file_path = r"C:\github\job_bot\input_output\human_review\resps_reqs_matching_trimmed\Blend_Director__AI_Strategy_crosstab_trimmed.json"

data = load_and_clean_json_file(file_path)

word_count = sum(len(v.split()) for v in data.values() if isinstance(v, str))

print("Trimmed and Condensed by GPT:")
print(f"Word count: {word_count}")

2025-03-19 17:46:34,930 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\human_review\resps_reqs_matching_trimmed\Blend_Director__AI_Strategy_crosstab_trimmed.json


Trimmed and Condensed by GPT:
Word count: 0


## Blend Job Posting

### Dataframe format - prioritized edited responsibilities manually

In [ ]:
company = "Blend"

file_path = r"C:\github\job_bot\input_output\human_review\resps_reqs_matching\reviewed_matchings\Blend_Director__AI_Strategy_crosstab_reviewed.xlsx"
sheet_name = "Crosstab"

wb, sheet = load_excel_sheet(file_path, sheet_name)
df = get_underlined_cells(sheet)


cleaned_df = clean_and_remove_rows(df)
final_df = rename_dataframe_columns(cleaned_df)

print(company)

pd.set_option("display.max_columns", None)
display(final_df)

wb.close()

Blend


,original_responsibility,edited_responsibility_1
responsibility_key,,
0.responsibilities.0,Conversational AI & NLP: Engineered the core d...,Engineered the core dialogue system to generat...
0.responsibilities.1,Thought Generation & Processing: Designed pipe...,Led the design and deployment of AI-powered so...
0.responsibilities.2,Automated Evaluation & Adaptation: Developed a...,Led the development of AI-powered systems that...
0.responsibilities.3,State & Topic Management: Built a stateful tra...,Led the design and implementation of a statefu...
0.responsibilities.4,Asynchronous AI Integration: Optimized API per...,Led the optimization of high-performance AI sy...
1.responsibilities.0,Provided strategic insights to a major global ...,Led strategic initiatives to optimize the serv...
1.responsibilities.1,Assisted a U.S.-based international services p...,NaN
1.responsibilities.2,Co-authored an industry-recognized report on M...,NaN
1.responsibilities.3,Enhanced data quality and consistency by integ...,Led comprehensive financial analysis and proce...


#### JSON format: Original

In [ ]:
file_path = r"C:\github\job_bot\input_output\human_review\resps_reqs_matching\reviewed_matchings\Blend_Director__AI_Strategy_crosstab_reviewed.xlsx"
sheet_name = "Crosstab"

wb, sheet = load_excel_sheet(file_path, sheet_name)
df = get_underlined_cells(sheet)
cleaned_df = clean_and_remove_rows(df)
final_df = rename_dataframe_columns(cleaned_df)

final_df.drop("edited_responsibility_1", axis=1, inplace=True)
# print(final_df)

json_string = final_df.to_json()
json_data = json.loads(json_string)  # Convert string to dictionary

print("JSON: original")
display_json_pretty(json_data)

wb.close()

JSON: original


```json
{
  "original_responsibility": {
    "0.responsibilities.0": "Conversational AI & NLP: Engineered the core dialogue system to generate dynamic questions,
evaluate responses, and maintain conversation flow.",
    "0.responsibilities.1": "Thought Generation & Processing: Designed pipelines for hierarchical idea expansion,
clustering, and ranking AI-generated thoughts.",
    "0.responsibilities.2": "Automated Evaluation & Adaptation: Developed an AI-based evaluator to assess user
responses, generate adaptive follow-ups, and track discussion depth.",
    "0.responsibilities.3": "State & Topic Management: Built a stateful tracking system to guide conversations, handle
topic transitions, and prevent redundancy.",
    "0.responsibilities.4": "Asynchronous AI Integration: Optimized API performance for OpenAI GPT-4 & Anthropic Claude
using AsyncIO, improving efficiency and scalability.",
    "1.responsibilities.0": "Provided strategic insights to a major global IT vendor, optimizing their service partner
ecosystem in Asia Pacific for improved local implementation outcomes.",
    "1.responsibilities.1": "Assisted a U.S.-based international services provider in its growth strategy by precisely
evaluating and scaling new engineering service opportunities in vital emerging markets.",
    "1.responsibilities.2": "Co-authored an industry-recognized report on M&A in the engineering services sector,
offering deep dives into deal sizes, capability gaps, and emerging opportunities, influencing strategic decisions in IT
and operational technology convergence.",
    "1.responsibilities.3": "Enhanced data quality and consistency by integrating thorough financial analysis,
standardizing methodologies, and conducting in-depth vendor engagements.",
    "1.responsibilities.4": "Achieved over 40% centralization of tasks to an offshore team in India, optimizing resource
allocation.",
    "1.responsibilities.5": "Developed Python tools to automate and accelerate internal processes, cutting report
preparation and data analysis time by over 40%.",
    "1.responsibilities.6": "Collaborated with the engineering services research team to pioneer the engineering
services tracker, authored influential publications on market forecasts, the impact of COVID-19 on services, and trends
in M&A within the engineering services industry.",
    "1.responsibilities.7": "Collaborated with analyst teams across the globe (US, Canada, Latin America, Europe, MEA,
APAC) to ensure data quality, meeting deadlines, sharing knowledge/best practices/methodology, and procure new tools.",
    "2.responsibilities.0": "Full P&L management including budgeting, HR, vendors, partnerships, research, and business
development: expanded the programs' bookings by more than 50%.",
    "2.responsibilities.1": "Expanded and managed a global research team of more than 15 (US, India, and Mexico).",
    "2.responsibilities.2": "Led an external software development team to build and implement new tools.",
    "2.responsibilities.3": "Championed new technology projects using ML, NLP, chatbot, ontology, web-scraping, API, UX
(User Experience). First in IDC to implement ML (machine learning). First to launch a chatbot. First to implement API
(application programming interface). First to build an ontology.",
    "2.responsibilities.4": "Led a team to develop automated tools in Python, reducing report preparation time by 40%.",
    "2.responsibilities.5": "Advised services firms on deal pursuit and sales orchestration strategies.",
    "2.responsibilities.6": "Advised software vendors on services partnership strategy.",
    "2.responsibilities.7": "Authored reports, blogs, presentations, & custom researches in go-to-market strategy, deal
signing analysis, renewal analysis, buyer studies, technology adoptions (cloud, AI, ML, digital, etc.), and industry
trend analysis.",
    "3.responsibilities.0": "Delivered quarterly webinars on outsourcing/managed services signing trends.",
    "3.responsibilities.1": "Authored 7 to 10 pursuit strategy reports and industry trend research documents.",
    "4.responsibilities.0": "Designed and architected a complex company database with 100K+ unique records and
integrating DnB API and internal databases, which reduced data duplication by 50%.",
    "4.responsibilities.1": "Oversaw two Appian implementations that increased team productivity by 20 to 30%.",
    "4.responsibilities.2": "Managed daily content operations, including leading a team of offshore and nearshore
content team, as well as other sales and research related activities.",
    "4.responsibilities.3": "Analyzed and Modeled 20 to 30 IT vendors' financials and over 1,500 services contracts.",
    "4.responsibilities.4": "Managed three major data integration projects critical to the launch of the new platform.",
    "4.responsibilities.5": "Promoted from Senior Research Analyst to Research Manager in February 2007.",
    "5.responsibilities.0": "Researched market dynamics in the Web Services Security market to support product
strategy."
  }
}
```

#### JSON format: edited by LLM

In [ ]:
file_path = r"C:\github\job_bot\input_output\human_review\resps_reqs_matching\reviewed_matchings\Blend_Director__AI_Strategy_crosstab_reviewed.xlsx"
sheet_name = "Crosstab"

wb, sheet = load_excel_sheet(file_path, sheet_name)

df = get_underlined_cells(sheet)
cleaned_df = clean_and_remove_rows(df)

final_df = rename_dataframe_columns(cleaned_df)
final_df.drop("original_responsibility", axis=1, inplace=True)


# print(final_df)
json_string = final_df.to_json()
# json_data = json.loads(json_string)  # Convert string to dictionary

print("JSON String:")

# # Format JSON for readability
# formatted_json = format_json_readable(json_data, wrap_width=120)

# # Display with Markdown to prevent horizontal scrolling
# display(Markdown(f"```json\n{formatted_json}\n```"))


display_json_pretty(json_string)
wb.close()

JSON String:


```json
{
  "edited_responsibility_1": {
    "0.responsibilities.0": "Engineered the core dialogue system to generate dynamic questions, evaluate responses, and
maintain conversation flow, driving substantial business value and revenue growth through strategic AI initiatives.",
    "0.responsibilities.1": "Led the design and deployment of AI-powered solutions to drive ideation, thought
generation, and business innovation. Developed hierarchical idea expansion, clustering, and ranking pipelines to unlock
new insights and opportunities.",
    "0.responsibilities.2": "Led the development of AI-powered systems that analyze user feedback, generate dynamic
responses, and monitor conversation depth. Collaborative leadership approach focused on continuous improvement and team
development.",
    "0.responsibilities.3": "Led the design and implementation of a stateful tracking system to facilitate seamless
topic management and enhance conversational efficiency. Leveraged analytical and problem-solving skills to guide
discussions, enable smooth transitions, and ensure overall conversational effectiveness.",
    "0.responsibilities.4": "Led the optimization of high-performance AI systems, leveraging cutting-edge technologies
like AsyncIO to enhance the scalability and efficiency of large language models such as OpenAI GPT-4 and Anthropic
Claude. Brings a collaborative leadership approach and a growth-oriented mindset to drive continuous improvement and
innovation.",
    "1.responsibilities.0": "Led strategic initiatives to optimize the service partner ecosystem of a leading global IT
vendor in the Asia Pacific region, driving improved local implementation outcomes and demonstrating strong analytical
and problem-solving capabilities.",
    "1.responsibilities.1": null,
    "1.responsibilities.2": null,
    "1.responsibilities.3": "Led comprehensive financial analysis and process standardization initiatives, driving data
integrity and uniformity through extensive vendor collaborations. Demonstrated a collaborative leadership style focused
on continuous improvement and growth.",
    "1.responsibilities.4": "Led centralized task management initiatives, achieving over 40% offshoring to leverage
global talent and drive operational efficiency. Cultivated a collaborative, growth-oriented team environment.",
    "1.responsibilities.5": null,
    "1.responsibilities.6": "Pioneered the engineering services tracker as part of the research team. Authored
influential publications on market forecasts, the impact of COVID-19 on services, and trends in M&A within the
engineering services industry. Demonstrated strong communication and collaboration skills, adept at working effectively
with cross-functional teams.",
    "1.responsibilities.7": "Led global analyst teams to ensure data integrity, meet critical deadlines, share
expertise, and procure innovative tools. Drove successful implementation of AI solutions in a business environment.",
    "2.responsibilities.0": "Seasoned professional who led full P&L management, driving significant growth by expanding
program bookings over 50%. Demonstrated expertise in budgeting, HR, vendor relations, partnerships, research, and
business development. Possesses a deep understanding of AI technologies and their business applications.",
    "2.responsibilities.1": "Led and grew a multicultural research team with a collaborative, growth-focused approach.",
    "2.responsibilities.2": "Led external software development teams to conceptualize and implement cutting-edge
solutions, leveraging strong communication and collaboration abilities to drive cross-functional alignment.",
    "2.responsibilities.3": "Innovative technology leader who championed transformative solutions leveraging cutting-
edge tools like machine learning, natural language processing, chatbots, ontologies, web scraping, and APIs. Drove
substantial business value and revenue growth as the first in the organization to implement these advanced technologies,
including launching a chatbot and building an ontology.",
    "2.responsibilities.4": "Led a team in developing automated Python tools, driving a 40% reduction in report
preparation time. Demonstrated a collaborative approach focused on continuous improvement and a growth-oriented
mindset.",
    "2.responsibilities.5": "Advised services firms on deal pursuit and sales orchestration strategies, leveraging
excellent communication and interpersonal skills to collaborate effectively with cross-functional teams.",
    "2.responsibilities.6": "Leveraged strong communication and interpersonal skills to develop strategic software
partnerships by collaborating effectively across diverse teams.",
    "2.responsibilities.7": "Led the creation of impactful content, including reports, blogs, presentations, and custom
research, to drive strategic initiatives across areas such as go-to-market strategy, deal analysis, customer insights,
and technology trends. Demonstrated a collaborative leadership approach and a growth-oriented mindset.",
    "3.responsibilities.0": null,
    "3.responsibilities.1": "Authored strategic reports and industry analyses, demonstrating strong communication and
collaboration skills.",
    "4.responsibilities.0": "Designed and implemented a comprehensive enterprise-level database system, integrating
multiple internal and external data sources to enhance data integrity and reduce duplication by a significant margin.
Demonstrated strong communication and technical expertise in delivering this complex project, working effectively with
cross-functional teams.",
    "4.responsibilities.1": "Led successful Appian implementations that enhanced team productivity.",
    "4.responsibilities.2": "Led cross-functional content teams and oversaw daily content operations, including sales
and research-focused initiatives. Demonstrated proficiency in analytical problem-solving and data-driven decision-
making.",
    "4.responsibilities.3": "Analyzed financials of 20-30 IT service providers and managed a portfolio of over 1,500
contracts, leveraging advanced analytical and financial modeling skills to provide in-depth insights. Demonstrated a
collaborative leadership approach and a growth-oriented mindset in driving strategic decision-making.",
    "4.responsibilities.4": "Led the successful delivery of three mission-critical data integration projects that
enabled the launch of a new enterprise-level platform. Drove large-scale AI strategy and implementation initiatives at
the enterprise level.",
    "4.responsibilities.5": null,
    "5.responsibilities.0": null
  }
}
```

#### Final trim & edit by GPT

In [ ]:
file_path = r"C:\github\job_bot\input_output\human_review\resps_reqs_matching_trimmed\Blend_Director__AI_Strategy_crosstab_trimmed.json"

data = load_and_clean_json_file(file_path)

word_count = 0


# Iterate through the values of the JSON object
def count_words(data):
    word_count = 0
    for v in data.values():
        if isinstance(v, dict):
            word_count += count_words(v)
        elif isinstance(v, str):
            word_count += len(v.split())
    return word_count


word_count = count_words(data)


print("Trimmed and Condensed by GPT:")
print(f"Word count: {word_count}")


# word_count = sum(len(v.split()) for v in data.values())


display_json_pretty(file_path)

Trimmed and Condensed by GPT:
Word count: 273


```json
{
  "edited_responsibility_1": {
    "0.responsibilities.0": "Developed AI-driven dialogue systems optimizing user engagement and revenue growth.",
    "0.responsibilities.1": "Designed AI solutions for ideation and business innovation, leveraging hierarchical
clustering and ranking algorithms.",
    "0.responsibilities.2": "Built AI-powered feedback analysis systems to enhance response generation and conversation
tracking.",
    "0.responsibilities.3": "Implemented stateful tracking for seamless topic management, improving discussion flow and
engagement.",
    "0.responsibilities.4": "Optimized large-scale AI systems using AsyncIO for enhanced scalability and efficiency.",
    "1.responsibilities.0": "Led strategic initiatives optimizing IT vendor ecosystems in APAC, improving implementation
outcomes.",
    "1.responsibilities.3": "Standardized financial analysis and vendor collaborations, improving data integrity and
decision-making.",
    "1.responsibilities.4": "Achieved 40% operational offshoring to enhance global talent utilization and efficiency.",
    "1.responsibilities.6": "Developed an engineering services tracker, publishing key reports on market trends and M&A
activity.",
    "1.responsibilities.7": "Led global analyst teams to maintain data integrity, meet deadlines, and integrate AI-
driven business insights.",
    "2.responsibilities.0": "Managed full P&L, driving 50% program growth and leading strategy in budgeting, HR, and
vendor relations.",
    "2.responsibilities.1": "Grew a multicultural research team, fostering a collaborative, high-performance
environment.",
    "2.responsibilities.2": "Directed external development teams to design and implement cutting-edge AI solutions.",
    "2.responsibilities.3": "Pioneered ML, NLP, chatbots, and ontologies, significantly advancing AI adoption within the
organization.",
    "2.responsibilities.4": "Led automation initiatives in Python, reducing report preparation time by 40%.",
    "2.responsibilities.5": "Advised services firms on strategic deal pursuit and sales orchestration.",
    "2.responsibilities.6": "Developed strategic software partnerships through cross-functional collaboration.",
    "2.responsibilities.7": "Created high-impact content including reports, blogs, and presentations to drive business
insights.",
    "3.responsibilities.1": "Authored strategic industry reports, delivering high-impact insights to executive
stakeholders.",
    "4.responsibilities.0": "Designed an enterprise-level database, integrating multiple data sources to enhance data
integrity.",
    "4.responsibilities.1": "Led Appian implementations improving team productivity.",
    "4.responsibilities.2": "Managed cross-functional content teams overseeing research and sales-focused content
strategies.",
    "4.responsibilities.3": "Analyzed financials of 20+ IT service providers, managing a portfolio of 1,500+
contracts.",
    "4.responsibilities.4": "Delivered three critical data integration projects, enabling enterprise-level AI adoption."
  }
}
```

In [ ]:
file_path = r"C:\github\job_bot\input_output\human_review\resps_reqs_matching_trimmed\Blend_Director__AI_Strategy_crosstab_trimmed.json"

data = load_and_clean_json_file(file_path)

word_count = sum(len(v.split()) for v in data.values() if isinstance(v, str))

print("Trimmed and Condensed by GPT:")
print(f"Word count: {word_count}")

Trimmed and Condensed by GPT:
Word count: 0


## Glean Job Posting

### Dataframe format - prioritized edited responsibilities manually

In [14]:
import xlwings as xw


company = "Glean"
glean_file = r"C:\github\job_bot\input_output\human_review\resps_reqs_matching\reviewed_matchings\Glean_Head_of_Competitive_Intelligence_crosstab.xlsx"

# wb = xw.Book(glean_file)
sheet_name = "Crosstab"

wb, sheet = load_excel_sheet(file_path=glean_file, sheet_name=sheet_name)
df = get_underlined_cells(sheet)


cleaned_df = clean_and_remove_rows(df)
final_df = rename_dataframe_columns(cleaned_df)

print(company)

pd.set_option("display.max_columns", None)
display(final_df)

wb.close()

Glean


,original_responsibility,edited_responsibility_1
responsibility_key,,
0.responsibilities.0,Conversational AI & NLP: Engineered the core d...,Conversational AI & NLP: Engineered a dynamic ...
0.responsibilities.1,Thought Generation & Processing: Designed pipe...,Thought Generation & Processing: Expertly desi...
0.responsibilities.2,Automated Evaluation & Adaptation: Developed a...,Developed an AI-based evaluator in a software ...
0.responsibilities.3,State & Topic Management: Built a stateful tra...,Built a stateful tracking system for guiding c...
0.responsibilities.4,Asynchronous AI Integration: Optimized API per...,Proactive in AI Integration: Optimized API per...
1.responsibilities.0,Provided strategic insights to a major global ...,Optimized service partner ecosystems for a maj...
1.responsibilities.1,Assisted a U.S.-based international services p...,Assisted a U.S.-based international services p...
1.responsibilities.2,Co-authored an industry-recognized report on M...,Co-authored an industry-recognized report on M...
1.responsibilities.3,Enhanced data quality and consistency by integ...,Enhanced data quality and consistency by integ...


#### JSON format: Original

In [ ]:
file_path = glean_file
sheet_name = "Crosstab"

wb, sheet = load_excel_sheet(file_path, sheet_name)
df = get_underlined_cells(sheet)
cleaned_df = clean_and_remove_rows(df)
final_df = rename_dataframe_columns(cleaned_df)

final_df.drop("edited_responsibility_1", axis=1, inplace=True)

json_string = final_df.to_json()
json_data = json.loads(json_string)  # Convert string to dictionary

print("JSON: original")
display_json_pretty(json_data)

wb.close()

JSON: original


```json
{
  "original_responsibility": {
    "0.responsibilities.0": "Conversational AI & NLP: Engineered the core dialogue system to generate dynamic questions,
evaluate responses, and maintain conversation flow.",
    "0.responsibilities.1": "Thought Generation & Processing: Designed pipelines for hierarchical idea expansion,
clustering, and ranking AI-generated thoughts.",
    "0.responsibilities.2": "Automated Evaluation & Adaptation: Developed an AI-based evaluator to assess user
responses, generate adaptive follow-ups, and track discussion depth.",
    "0.responsibilities.3": "State & Topic Management: Built a stateful tracking system to guide conversations, handle
topic transitions, and prevent redundancy.",
    "0.responsibilities.4": "Asynchronous AI Integration: Optimized API performance for OpenAI GPT-4 & Anthropic Claude
using AsyncIO, improving efficiency and scalability.",
    "1.responsibilities.0": "Provided strategic insights to a major global IT vendor, optimizing their service partner
ecosystem in Asia Pacific for improved local implementation outcomes.",
    "1.responsibilities.1": "Assisted a U.S.-based international services provider in its growth strategy by precisely
evaluating and scaling new engineering service opportunities in vital emerging markets.",
    "1.responsibilities.2": "Co-authored an industry-recognized report on M&A in the engineering services sector,
offering deep dives into deal sizes, capability gaps, and emerging opportunities, influencing strategic decisions in IT
and operational technology convergence.",
    "1.responsibilities.3": "Enhanced data quality and consistency by integrating thorough financial analysis,
standardizing methodologies, and conducting in-depth vendor engagements.",
    "1.responsibilities.4": "Achieved over 40% centralization of tasks to an offshore team in India, optimizing resource
allocation.",
    "1.responsibilities.5": "Developed Python tools to automate and accelerate internal processes, cutting report
preparation and data analysis time by over 40%.",
    "1.responsibilities.6": "Collaborated with the engineering services research team to pioneer the engineering
services tracker, authored influential publications on market forecasts, the impact of COVID-19 on services, and trends
in M&A within the engineering services industry.",
    "1.responsibilities.7": "Collaborated with analyst teams across the globe (US, Canada, Latin America, Europe, MEA,
APAC) to ensure data quality, meeting deadlines, sharing knowledge/best practices/methodology, and procure new tools.",
    "2.responsibilities.0": "Full P&L management including budgeting, HR, vendors, partnerships, research, and business
development: expanded the programs' bookings by more than 50%.",
    "2.responsibilities.1": "Expanded and managed a global research team of more than 15 (US, India, and Mexico).",
    "2.responsibilities.2": "Led an external software development team to build and implement new tools.",
    "2.responsibilities.3": "Championed new technology projects using ML, NLP, chatbot, ontology, web-scraping, API, UX
(User Experience). First in IDC to implement ML (machine learning). First to launch a chatbot. First to implement API
(application programming interface). First to build an ontology.",
    "2.responsibilities.4": "Led a team to develop automated tools in Python, reducing report preparation time by 40%.",
    "2.responsibilities.5": "Advised services firms on deal pursuit and sales orchestration strategies.",
    "2.responsibilities.6": "Advised software vendors on services partnership strategy.",
    "2.responsibilities.7": "Authored reports, blogs, presentations, & custom researches in go-to-market strategy, deal
signing analysis, renewal analysis, buyer studies, technology adoptions (cloud, AI, ML, digital, etc.), and industry
trend analysis.",
    "3.responsibilities.0": "Delivered quarterly webinars on outsourcing/managed services signing trends.",
    "3.responsibilities.1": "Authored 7 to 10 pursuit strategy reports and industry trend research documents.",
    "4.responsibilities.0": "Designed and architected a complex company database with 100K+ unique records and
integrating DnB API and internal databases, which reduced data duplication by 50%.",
    "4.responsibilities.1": "Oversaw two Appian implementations that increased team productivity by 20 to 30%.",
    "4.responsibilities.2": "Managed daily content operations, including leading a team of offshore and nearshore
content team, as well as other sales and research related activities.",
    "4.responsibilities.3": "Analyzed and Modeled 20 to 30 IT vendors' financials and over 1,500 services contracts.",
    "4.responsibilities.4": "Managed three major data integration projects critical to the launch of the new platform.",
    "4.responsibilities.5": "Promoted from Senior Research Analyst to Research Manager in February 2007.",
    "5.responsibilities.0": "Researched market dynamics in the Web Services Security market to support product
strategy."
  }
}
```

#### JSON format: edited by LLM

In [ ]:
file_path = glean_file
sheet_name = "Crosstab"

wb, sheet = load_excel_sheet(file_path, sheet_name)

df = get_underlined_cells(sheet)
cleaned_df = clean_and_remove_rows(df)

final_df = rename_dataframe_columns(cleaned_df)
final_df.drop("original_responsibility", axis=1, inplace=True)


# print(final_df)
json_string = final_df.to_json()
# json_data = json.loads(json_string)  # Convert string to dictionary

print(company)
print("JSON")

# # Format JSON for readability
# formatted_json = format_json_readable(json_data, wrap_width=120)

# # Display with Markdown to prevent horizontal scrolling
# display(Markdown(f"```json\n{formatted_json}\n```"))


display_json_pretty(json_string)
wb.close()

Glean
JSON String:


```json
{
  "edited_responsibility_1": {
    "0.responsibilities.0": "Conversational AI & NLP: Engineered a dynamic dialogue system that significantly enhanced
interaction capabilities and response assessment, crucial to my software development role, especially in technical B2B
SaaS products. This role entailed collaborating closely with developer teams, thereby broadening my understanding of
software development processes.",
    "0.responsibilities.1": "Thought Generation & Processing: Expertly designed and implemented advanced pipelines for
hierarchical idea expansion, clustering, and ranking of AI-generated thoughts. Effectively managed and completed
individual deliverables while concurrently building and developing a small team.",
    "0.responsibilities.2": "Developed an AI-based evaluator in a software development environment to assess user
interactions and generate adaptive responses, enhancing the capabilities of technical B2B SaaS products. Led cross-
functional teams in close collaboration, leveraging extensive experience with developer coordination to drive product
improvements.",
    "0.responsibilities.3": "Built a stateful tracking system for guiding conversations that integrates with essential
SaaS applications, ensuring cloud technology framework compatibility, data center alignment, and enterprise-level
security considerations.",
    "0.responsibilities.4": "Proactive in AI Integration: Optimized API performance for major AI models like OpenAI
GPT-4 and Anthropic Claude by employing AsyncIO, enhancing efficiency and scalability. Demonstrated a hands-on approach
and strategic thinking in technical improvements.",
    "1.responsibilities.0": "Optimized service partner ecosystems for a major IT vendor in Asia Pacific by balancing and
prioritizing competing opportunities, customer needs, and business priorities, leading to enhanced local implementation
outcomes.",
    "1.responsibilities.1": "Assisted a U.S.-based international services provider by strategically evaluating and
expanding engineering service opportunities in vital emerging markets. Demonstrated exceptional written and verbal
communication skills by simplifying complex concepts and effectively influencing stakeholders. Maintained meticulous
attention to detail throughout the project.",
    "1.responsibilities.2": "Co-authored an industry-recognized report on M&A in the engineering services sector that
simplified complex deal sizes and capability gaps while identifying emerging opportunities. This work influenced
strategic decisions across IT and operational technology convergence, demonstrating exceptional written and verbal
communication skills and meticulous attention to detail.",
    "1.responsibilities.3": "Enhanced data quality and consistency by integrating thorough financial analysis,
standardizing methodologies, and conducting in-depth vendor engagements. Demonstrated comfort with hands-on tasks and
completing individual deliverables, while also leading and growing a small team.",
    "1.responsibilities.4": "Achieved over 40% centralization of tasks to an offshore team in India, successfully
driving individual deliverables while building and developing a small team's capabilities.",
    "1.responsibilities.5": "Developed Python tools to automate and accelerate internal processes, cutting report
preparation and data analysis time by over 40%. Collaborated extensively with software development teams, contributing
to the development of technical B2B SaaS products and gaining deep familiarity with software development processes.",
    "1.responsibilities.6": "With over a decade of experience in product marketing and technical sales, collaborated
with the engineering services research team to pioneer the engineering services tracker. Authored influential
publications on market forecasts, the impact of COVID-19 on services, and M&A trends within the industry.",
    "1.responsibilities.7": "Collaborated with global analyst teams across the US, Canada, Latin America, Europe, MEA,
and APAC to enhance data quality and meet project deadlines. Actively facilitated knowledge sharing and the procurement
of new tools, while driving the completion of individual deliverables and contributing to the growth and development of
a small team.",
    "2.responsibilities.0": "Managed full P&L responsibilities, incorporating budget oversight, human resources
coordination, and vendor relations, while building strategic partnerships. Directed research and led business
development efforts, expanding program bookings by over 50%. Demonstrated exceptional written and verbal communication
skills, adeptly simplifying complex concepts to engage and persuade stakeholders, all with meticulous attention to
detail.",
    "2.responsibilities.1": "Expanded and managed a diverse research team across multiple countries, skillfully
balancing and prioritizing competing business priorities and customer needs.",
    "2.responsibilities.2": "Led an external software development team to create and implement innovative B2B SaaS
software tools, demonstrating extensive involvement in software development processes and effective collaboration with
developer teams.",
    "2.responsibilities.3": "Championed the integration of advanced technology initiatives at IDC, including machine
learning, natural language processing, chatbots, and API integrations, with a focus on enhancing user experience.
Spearheaded the adoption of these technologies alongside modern SaaS applications, data center optimizations, and cloud
technologies, ensuring compliance with enterprise security standards.",
    "2.responsibilities.4": "Led a team in developing Python-based automation tools, slashing report preparation time by
40%, which enhanced operational efficiency and exemplified a proactive, hands-on approach to strategic execution.",
    "2.responsibilities.5": "Advised service firms in developing and executing deal pursuit strategies and sales
orchestration, skillfully managed individual deliverables while concurrently expanding a small team.",
    "2.responsibilities.6": "Advised software vendors extensively to develop partnership strategies, specializing in
technical B2B SaaS products and closely collaborated with developer teams to enhance familiarity with software
development processes.",
    "2.responsibilities.7": "Authored various reports, blogs, presentations, and custom research focused on go-to-market
strategies, deal signing, renewal processes, buyer behavior, and technology adoption trends including cloud, AI, ML, and
digital transformations. Demonstrated exceptional written and verbal communication skills by simplifying complex
concepts for a diverse range of stakeholders and consistently maintained impeccable attention to detail.",
    "3.responsibilities.0": "Delivered quarterly webinars on outsourcing and managed services trends, demonstrating
stellar verbal and written communication skills, effectively influencing stakeholders, and showcasing impeccable
attention to detail.",
    "3.responsibilities.1": "Authored multiple pursuit strategy reports and industry trend research documents,
demonstrating a strong capacity for hands-on tasks and successfully delivering individual contributions while also
enhancing the development and growth of a small team.",
    "4.responsibilities.0": "Designed and architected a sophisticated company database encompassing over 100,000 unique
records, seamlessly integrating both external and internal data sources. This initiative, involving crucial SaaS
applications along with data center and cloud technologies, significantly bolstered data integrity and slashed
redundancy by 50%, while addressing key enterprise security considerations.",
    "4.responsibilities.1": "Oversaw multiple Appian software implementations, boosting team productivity significantly
and demonstrating deep knowledge of software development processes. Engaged closely with developer teams and interacted
extensively with technical B2B SaaS products.",
    "4.responsibilities.2": "Managed product marketing, product management, and technical sales for over a decade,
including overseeing content operations and leading diverse teams.",
    "4.responsibilities.3": "Analyzed key SaaS applications, data centers, cloud technologies, and enterprise security
by assessing financial data and modeling IT services contracts.",
    "4.responsibilities.4": "Managed multiple high-priority data integration projects, aligning effectively with
customer needs and business objectives for the successful launch of a new platform.",
    "4.responsibilities.5": null,
    "5.responsibilities.0": "Studied market dynamics in the Web Services Security industry to enhance competitive
marketing strategies."
  }
}
```

#### Final trim & edit by GPT

##### Create prompt

In [ ]:
from prompts.prompt_templates import TRIM_CONDENSE_FINAL_RESUME_WITH_JOB_DESC_PROMPT
from models.resume_job_description_io_models import JobFileMappings
from evaluation_optimization.create_mapping_file import load_mappings_model_from_json

# Set file path and url
file_path = glean_file
url = "https://boards.greenhouse.io/gleanwork/jobs/4425502005?source=LinkedIn"


# * Fetch edited resume in json format
sheet_name = "Crosstab"
wb, sheet = load_excel_sheet(file_path, sheet_name)

df = get_underlined_cells(sheet)
cleaned_df = clean_and_remove_rows(df)
final_df = rename_dataframe_columns(cleaned_df)
final_df.drop("original_responsibility", axis=1, inplace=True)

resps_json = final_df.to_json()


# * Fetch job_description
job_descriptions_file = JOB_DESCRIPTIONS_JSON_FILE
job_descriptions_dict = load_and_clean_json_file(job_descriptions_file)
job_description = job_descriptions_dict[url]["data"]

display_json_pretty(job_description)

# * Insert JSON into template
prompt_template = TRIM_CONDENSE_FINAL_RESUME_WITH_JOB_DESC_PROMPT
final_prompt = prompt_template.format(
    resume_json=resps_json, job_description_json=job_description
)

print(final_prompt)

2025-03-20 19:04:20,801 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\preprocessing\jobpostings.json


```json
{
  "url": "https://boards.greenhouse.io/gleanwork/jobs/4425502005?source=LinkedIn",
  "job_title": "Head of Competitive Intelligence",
  "company": "Glean",
  "location": "Palo Alto, CA",
  "salary_info": "The standard base salary range for this position is $145,000 - $190,000 annually. Compensation offered
will be determined by factors such as location, level, job-related knowledge, skills, and experience.",
  "posted_date": null,
  "content": {
    "About the role": "Glean is searching for an experienced Head of Competitive Intelligence. This individual will be
responsible for building the competitive intelligence function at Glean, working closely with Product Management
(feeding competitive information back into the product dev process), Product Marketing (influencing the outbound message
on competitors), and Field Enablement (training the Sales organization on how best to compete).",
    "What you will do and achieve": "Build Glean's Competitive Intelligence Marketing function and processes from
scratch, first acting as an individual contributor and eventually growing the function as required\nSegment Glean's
competitors into multiple tiers and provide up-to-date research and competitive intelligence on these
competitors\nTranslate competitive intelligence into competitive strategy, and execute field programs to effectively
engage and win vs. competitors\nCreate and execute a bi-directional competitive dialogue with the Sales organization,
working to train the field on competitive information and strategies, as well as collect up-to-date competitive
information from the field\nProduce \"day 0\" competitive responses on major news announcement days from
competitors\nEngage in competitive research, including directly hands-on evaluation of competitive products where
appropriate, competitive pricing research, and competitive field sales tactics\nDevise and execute an outbound
competitive strategy, effectively educating the market on Glean's advantages vs. competitors\nWork closely with the
Product Management, Product Marketing, Field Enablement, and Solutions Engineering functions at Glean",
    "Who you are": "BA/BS in engineering, business, or a related degree\n10+ years in product marketing, product
management, or technical sales\n3-5+ years management experience desired. Experience building and leading a team of
product and technical marketing professionals\nComfortable working \"hands on\" and driving/completing individual
deliverables while building-out a small team in parallel\nExperience in competitive marketing\nWorking knowledge of key
SaaS applications, data center and cloud technologies, and enterprise security considerations\nStella r written and
verbal communication, with experience simplifying concepts and influencing stakeholders, and impeccable attention to
detail\nA get-it-done attitude with ability to roll up your sleeves while driving strategy\nExperience working closely
with software development processes, familiarity with developer teams, and/or experience with technical B2B SaaS
products\nAbility to balance and prioritize competing opportunities, customer needs, and business priorities",
    "Benefits": "Competitive compensation\nMedical, Vision and Dental coverage\nFlexible work environment and time-off
policy\n401k\nCompany events\nA home office improvement stipend when you first join\nAnnual education stipend\nWellness
stipend\nHealthy lunches and dinners provided daily"
  }
}
```


You are an expert resume optimizer for senior-level tech and consulting roles.
Your task is to **trim, consolidate, and optimize** the provided JSON resume data while ensuring alignment with the job description provided.

### **Task Instructions**
1. **Trim & Condense:** Reduce the resume word count to approximately **500-600 words** while retaining essential details.
2. **Merge Related Content:** Combine overlapping or redundant bullets to enhance readability and conciseness.
3. **Enhance ATS Compliance:** Ensure the use of **strong action verbs**, **industry-specific keywords**, and **quantifiable achievements** (e.g., "% improvements," "increased revenue by X%," "cut processing time by Y%").
4. **Align with Job Description:** Ensure the resume highlights **key skills, experience, and qualifications** relevant to the job description.
5. **Ensure Consistency & Clarity:** Maintain a **structured, professional format**, ensuring smooth readability while preserving key impact points.
6.

##### LLM response output

In [ ]:
file_path = r"C:\github\job_bot\input_output\human_review\resps_reqs_matching_trimmed\Glean_Head_of_Competitive_Intelligence_crosstab_trimmed.json"
data = load_and_clean_json_file(file_path)

word_count = 0


# Iterate through the values of the JSON object
def count_words(data):
    word_count = 0
    for v in data.values():
        if isinstance(v, dict):
            word_count += count_words(v)
        elif isinstance(v, str):
            word_count += len(v.split())
    return word_count


word_count = count_words(data)


print("Trimmed and Condensed by GPT:")
print(f"Word count: {word_count}")
display_json_pretty(file_path)

2025-03-19 17:23:50,584 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\human_review\resps_reqs_matching_trimmed\Glean_Head_of_Competitive_Intelligence_crosstab_trimmed.json
2025-03-19 17:23:50,588 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\human_review\resps_reqs_matching_trimmed\Glean_Head_of_Competitive_Intelligence_crosstab_trimmed.json


Trimmed and Condensed by GPT:
Word count: 527


```json
{
  "edited_responsibility_1": {
    "0.responsibilities.0": "Developed and led AI-driven competitive intelligence frameworks, integrating NLP and data
science to enhance market strategy, supporting SaaS and enterprise software initiatives.",
    "0.responsibilities.1": "Designed and executed hierarchical thought generation and clustering pipelines to extract,
analyze, and synthesize competitive insights, aligning with product and market positioning.",
    "0.responsibilities.2": "Built an AI-based evaluator to assess market intelligence inputs and optimize decision-
making processes for go-to-market strategies and competitor benchmarking.",
    "0.responsibilities.3": "Developed and maintained a stateful conversation tracking system that integrates with SaaS
applications, ensuring real-time data capture and analysis for competitive insights.",
    "0.responsibilities.4": "Optimized API performance for major AI models (GPT-4, Claude) using AsyncIO, improving
efficiency and scalability of intelligence workflows.",
    "1.responsibilities.0": "Led competitive strategy for a major IT vendor in APAC, aligning business priorities with
customer needs to enhance local execution and market penetration.",
    "1.responsibilities.1": "Conducted in-depth evaluations of emerging engineering service markets, advising on growth
strategies and market entry, influencing executive decision-making.",
    "1.responsibilities.2": "Co-authored an industry-leading report on M&A trends in engineering services, analyzing
deal structures and emerging opportunities to inform strategic investments.",
    "1.responsibilities.3": "Standardized competitive intelligence methodologies, enhancing financial analysis
consistency and vendor benchmarking to support corporate strategy.",
    "1.responsibilities.4": "Achieved a 40% increase in operational efficiency by centralizing research functions to
offshore teams, improving data processing and reporting speed.",
    "1.responsibilities.5": "Developed Python automation tools that reduced report preparation and data analysis time by
over 40%, streamlining intelligence operations.",
    "1.responsibilities.6": "Authored key publications on engineering services market forecasts, SaaS adoption, and
competitive trends, influencing industry perspectives and investment decisions.",
    "1.responsibilities.7": "Collaborated with global analyst teams to refine data quality, enhance research
methodologies, and ensure timely delivery of market intelligence reports.",
    "2.responsibilities.0": "Managed full P&L responsibilities, overseeing budget, HR, and vendor relations while
driving strategic partnerships to expand competitive research capabilities.",
    "2.responsibilities.1": "Expanded and led a multi-country research team, balancing strategic objectives with market
intelligence deliverables.",
    "2.responsibilities.2": "Directed the development and deployment of innovative competitive intelligence tools,
integrating SaaS platforms, cloud technologies, and enterprise security frameworks.",
    "2.responsibilities.3": "Championed the adoption of AI-driven analytics, including machine learning, chatbots, and
NLP, to enhance competitor monitoring and strategic insights.",
    "2.responsibilities.4": "Led the automation of intelligence workflows, reducing manual effort in report generation
and increasing operational efficiency by 40%.",
    "2.responsibilities.5": "Advised service firms on sales strategy and competitive positioning, contributing to
multimillion-dollar contract wins.",
    "2.responsibilities.6": "Formulated strategic alliances with software vendors, optimizing competitive positioning
and product-market fit for technical B2B SaaS solutions.",
    "2.responsibilities.7": "Authored high-impact reports on go-to-market strategies, deal structuring, and competitive
landscapes, providing key insights for enterprise decision-makers.",
    "3.responsibilities.0": "Delivered quarterly webinars on competitive trends in outsourcing and managed services,
influencing senior executives and industry stakeholders.",
    "3.responsibilities.1": "Produced competitive intelligence reports and industry analyses, strengthening market
positioning and business strategy alignment.",
    "4.responsibilities.0": "Architected a competitive intelligence database with 100,000+ unique records, integrating
internal and external data sources to enhance insight accuracy and accessibility.",
    "4.responsibilities.1": "Led the implementation of workflow automation solutions, increasing research productivity
by 30% through streamlined intelligence processing.",
    "4.responsibilities.2": "Managed end-to-end competitive intelligence for enterprise SaaS and IT services, from data
acquisition to insight generation and strategic recommendation.",
    "4.responsibilities.3": "Analyzed SaaS applications, cloud adoption, and enterprise security trends to inform
competitive strategy and market positioning.",
    "4.responsibilities.4": "Directed high-priority intelligence projects, ensuring alignment with corporate business
objectives and strategic growth initiatives.",
    "5.responsibilities.0": "Researched market dynamics in Web Services Security, providing strategic insights to
support competitive positioning and product development."
  }
}
```

In [ ]:
file_path = r"C:\github\job_bot\input_output\human_review\resps_reqs_matching_trimmed\Glean_Head_of_Competitive_Intelligence_crosstab_trimmed.json"
data = load_and_clean_json_file(file_path)

edited_resps = data["edited_responsibility_1"]

formatted_text = "\n\n".join(f"{key}\n{value}" for key, value in edited_resps.items())

print(formatted_text)

2025-03-19 17:46:21,994 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\human_review\resps_reqs_matching_trimmed\Glean_Head_of_Competitive_Intelligence_crosstab_trimmed.json


0.responsibilities.0
Developed and led AI-driven competitive intelligence frameworks, integrating NLP and data science to enhance market strategy, supporting SaaS and enterprise software initiatives.

0.responsibilities.1
Designed and executed hierarchical thought generation and clustering pipelines to extract, analyze, and synthesize competitive insights, aligning with product and market positioning.

0.responsibilities.2
Built an AI-based evaluator to assess market intelligence inputs and optimize decision-making processes for go-to-market strategies and competitor benchmarking.

0.responsibilities.3
Developed and maintained a stateful conversation tracking system that integrates with SaaS applications, ensuring real-time data capture and analysis for competitive insights.

0.responsibilities.4
Optimized API performance for major AI models (GPT-4, Claude) using AsyncIO, improving efficiency and scalability of intelligence workflows.

1.responsibilities.0
Led competitive strategy for 

In [ ]:
file_path = r"C:\github\job_bot\input_output\human_review\resps_reqs_matching_trimmed\Blend_Director__AI_Strategy_crosstab_trimmed.json"

data = load_and_clean_json_file(file_path)

word_count = sum(len(v.split()) for v in data.values() if isinstance(v, str))

print("Trimmed and Condensed by GPT:")
print(f"Word count: {word_count}")

2025-03-19 17:46:34,930 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\human_review\resps_reqs_matching_trimmed\Blend_Director__AI_Strategy_crosstab_trimmed.json


Trimmed and Condensed by GPT:
Word count: 0


## SFDC Job Posting

### Get file name and url

In [4]:
import xlwings as xw
import pandas as pd
from IPython.display import display
from utils.get_file_names import get_file_names
from evaluation_optimization.create_mapping_file import load_mappings_model_from_json
from project_config import (
    ITERATE_1_OPENAI_DIR,
    mapping_file_name,
    REVIEWED_MATCHINGS_DIR,
)

# Get matching files
file_list = get_file_names(REVIEWED_MATCHINGS_DIR, full_path=True)

# Match company file (case-insensitive)
company = "salesforce"
company_file = next(
    (file for file in file_list if company.lower() in str(file).lower()), None
)

if not company_file:
    raise FileNotFoundError(f"No reviewed matchings file found for {company}")

# Load mapping file model
mapping_file = ITERATE_1_OPENAI_DIR / mapping_file_name
mapping_file_model = load_mappings_model_from_json(mapping_file)

# print(mapping_file_model)


# Get company URL(s) from mapping model
company_url = next(
    str(key) for key in mapping_file_model.root if company.lower() in str(key).lower()
)

if not company_url:
    print(f"⚠️ No URLs found for company '{company}' in mapping file.")

# Output
print(f"\n✅ {company} file:\n{company_file}")
print(f"\n🔗 {company} URL(s):{company_url}")

2025-04-03 14:13:38,587 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\evaluation_optimization\evaluation_optimization_by_openai\iteration_1\url_to_file_mapping.json
2025-04-03 14:13:38,588 - evaluation_optimization.create_mapping_file - INFO - Loaded and validated mapping file from C:\github\job_bot\input_output\evaluation_optimization\evaluation_optimization_by_openai\iteration_1\url_to_file_mapping.json



✅ salesforce file:
C:\github\job_bot\input_output\human_review\resps_reqs_matchings_reviewed\Salesforce_com_Vice_President__Product_Research___Insights_crosstab_reviewed.xlsx

🔗 salesforce URL(s):https://salesforce.wd12.myworkdayjobs.com/External_Career_Site/job/California---San-Francisco/Vice-President--Product-Research---Insights_JR279859?source=LinkedIn_Jobs


In [5]:
# Set file and url vars

if company_file is None:
    raise ValueError("company_file (sfdc) is None")

if company_url is None:
    raise ValueError("company_url (sfdc_url) is None")

sfdc_file = company_file
sfdc_url = company_url

print(f"{company} file and url parameter set.\n{company_file}\n{company_url}")

salesforce file and url parameter set.
C:\github\job_bot\input_output\human_review\resps_reqs_matchings_reviewed\Salesforce_com_Vice_President__Product_Research___Insights_crosstab_reviewed.xlsx
https://salesforce.wd12.myworkdayjobs.com/External_Career_Site/job/California---San-Francisco/Vice-President--Product-Research---Insights_JR279859?source=LinkedIn_Jobs


### Dataframe format, prioritized edited responsibilities manually

In [15]:
import xlwings as xw
import pandas as pd  # ✅ Required for pd.set_option and display
from IPython.display import display
from utils.get_file_names import get_file_names
from project_config import REVIEWED_MATCHINGS_DIR


# ✅ Get matching files
file_list = get_file_names(REVIEWED_MATCHINGS_DIR, full_path=True)

# ✅ Match company file
company = "salesforce"
company_file = next(
    (file for file in file_list if company.lower() in str(file).lower()), None
)

cross_tab_file = company_file

sheet_name = "Crosstab"

try:
    if not cross_tab_file:
        raise FileNotFoundError(f"No file found for company: {company}")

    df_all_data = pd.read_excel(cross_tab_file)

    wb, sheet = load_excel_sheet(file_path=cross_tab_file, sheet_name=sheet_name)
    df, coordinates = get_underlined_cells(sheet=sheet, return_coords=True)

except Exception as e:
    print(f"Error loading job posting: {e}")
    df = None


# df is your DataFrame
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

# ✅ Continue only if DataFrame loaded successfully
if df is not None:
    cleaned_df = clean_and_remove_rows(df)
    final_df = rename_dataframe_columns(cleaned_df)

    print(company)
    print(cross_tab_file)
    # pd.set_option("display.max_columns", None)
    display(final_df)

    wb.close()

pd.set_option("display.max_colwidth", 90)  # or a large number like 200

# pyright: ignore[reportInvalidTypeForm]
summary_df = summarize_underlined_column_frequencies(
    df_all_data=df_all_data,
    underlined_coords=coordinates,
)
print(f"Requirements Balance: \n{summary_df}")
pd.reset_option("all")

salesforce
C:\github\job_bot\input_output\human_review\resps_reqs_matchings_reviewed\Salesforce_com_Vice_President__Product_Research___Insights_crosstab_reviewed.xlsx


,original_responsibility,edited_responsibility_1
responsibility_key,,
0.responsibilities.0,Conversational AI & NLP: Engineered the core d...,Conversational AI & NLP: Passionately engineer...
0.responsibilities.1,Thought Generation & Processing: Designed pipe...,"Designed AI-driven processes that generate, ex..."
0.responsibilities.2,Automated Evaluation & Adaptation: Developed a...,Developed an AI system passionate about enhanc...
0.responsibilities.3,State & Topic Management: Built a stateful tra...,Built and executed a stateful tracking system ...
0.responsibilities.4,Asynchronous AI Integration: Optimized API per...,Enhanced AI API performance for OpenAI GPT-4 &...
1.responsibilities.0,Provided strategic insights to a major global ...,Executed strategic research to enhance busines...
1.responsibilities.1,Assisted a U.S.-based international services p...,Assisted a U.S.-based international services p...
1.responsibilities.2,Co-authored an industry-recognized report on M...,Co-authored a comprehensive analysis on merger...
1.responsibilities.3,Enhanced data quality and consistency by integ...,Enhanced data quality and consistency by colla...


Requirements Balance: 
                                                                                 Column Name  Underline Count  Frequency (%)
0    Passionate about understanding and solving customer issues through data-driven insights                5           17.9
4        Strong leadership and management expertise with executive-level presentation skills                5           17.9
2                                  Expertise in executing research in a business environment                4           14.3
3  15+ years of experience leading research in product development, user experience, or t...                4           14.3
5                                  Collaborative mindset to work with cross-functional teams                4           14.3
1           Proven track record of delivering insight work with measurable business outcomes                3           10.7
6  Experience navigating complex organizational issues using collaborative and directive ...          

C:\Users\xzhan\AppData\Local\Temp\ipykernel_26356\1369930956.py:59: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option("all")
C:\Users\xzhan\AppData\Local\Temp\ipykernel_26356\1369930956.py:59: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option("all")


### JSON format: Original, prioritized edited responsibilities manually

In [17]:
import xlwings as xw
import pandas as pd  # ✅ Required for pd.set_option and display
from IPython.display import display
from utils.get_file_names import get_file_names
from project_config import REVIEWED_MATCHINGS_DIR


# ✅ Get matching files
file_list = get_file_names(REVIEWED_MATCHINGS_DIR, full_path=True)

# ✅ Match company file
company = "salesforce"
company_file = next(
    (file for file in file_list if company.lower() in str(file).lower()), None
)

# ✅ Match company file
company = "salesforce"
company_file = next(
    (file for file in file_list if company.lower() in str(file).lower()), None
)

file_path = company_file
sheet_name = "Crosstab"

if file_path is not None:
    wb, sheet = load_excel_sheet(file_path, sheet_name)
    df = get_underlined_cells(sheet)
    cleaned_df = clean_and_remove_rows(df)
    final_df = rename_dataframe_columns(cleaned_df)
else:
    raise FileExistsError

final_df.drop("edited_responsibility_1", axis=1, inplace=True)

json_string = final_df.to_json()
json_data = json.loads(json_string)  # Convert string to dictionary

print("JSON: original")
display_json_pretty(json_data)

wb.close()

JSON: original


```json
{
  "original_responsibility": {
    "0.responsibilities.0": "Conversational AI & NLP: Engineered the core dialogue system to generate dynamic questions,
evaluate responses, and maintain conversation flow.",
    "0.responsibilities.1": "Thought Generation & Processing: Designed pipelines for hierarchical idea expansion,
clustering, and ranking AI-generated thoughts.",
    "0.responsibilities.2": "Automated Evaluation & Adaptation: Developed an AI-based evaluator to assess user
responses, generate adaptive follow-ups, and track discussion depth.",
    "0.responsibilities.3": "State & Topic Management: Built a stateful tracking system to guide conversations, handle
topic transitions, and prevent redundancy.",
    "0.responsibilities.4": "Asynchronous AI Integration: Optimized API performance for OpenAI GPT-4 & Anthropic Claude
using AsyncIO, improving efficiency and scalability.",
    "1.responsibilities.0": "Provided strategic insights to a major global IT vendor, optimizing their service partner
ecosystem in Asia Pacific for improved local implementation outcomes.",
    "1.responsibilities.1": "Assisted a U.S.-based international services provider in its growth strategy by precisely
evaluating and scaling new engineering service opportunities in vital emerging markets.",
    "1.responsibilities.2": "Co-authored an industry-recognized report on M&A in the engineering services sector,
offering deep dives into deal sizes, capability gaps, and emerging opportunities, influencing strategic decisions in IT
and operational technology convergence.",
    "1.responsibilities.3": "Enhanced data quality and consistency by integrating thorough financial analysis,
standardizing methodologies, and conducting in-depth vendor engagements.",
    "1.responsibilities.4": "Achieved over 40% centralization of tasks to an offshore team in India, optimizing resource
allocation.",
    "1.responsibilities.5": "Developed Python tools to automate and accelerate internal processes, cutting report
preparation and data analysis time by over 40%.",
    "1.responsibilities.6": "Collaborated with the engineering services research team to pioneer the engineering
services tracker, authored influential publications on market forecasts, the impact of COVID-19 on services, and trends
in M&A within the engineering services industry.",
    "1.responsibilities.7": "Collaborated with analyst teams across the globe (US, Canada, Latin America, Europe, MEA,
APAC) to ensure data quality, meeting deadlines, sharing knowledge/best practices/methodology, and procure new tools.",
    "2.responsibilities.0": "Full P&L management including budgeting, HR, vendors, partnerships, research, and business
development: expanded the programs' bookings by more than 50%.",
    "2.responsibilities.1": "Expanded and managed a global research team of more than 15 (US, India, and Mexico).",
    "2.responsibilities.2": "Led an external software development team to build and implement new tools.",
    "2.responsibilities.3": "Championed new technology projects using ML, NLP, chatbot, ontology, web-scraping, API, UX
(User Experience). First in IDC to implement ML (machine learning). First to launch a chatbot. First to implement API
(application programming interface). First to build an ontology.",
    "2.responsibilities.4": "Led a team to develop automated tools in Python, reducing report preparation time by 40%.",
    "2.responsibilities.5": "Advised services firms on deal pursuit and sales orchestration strategies.",
    "2.responsibilities.6": "Advised software vendors on services partnership strategy.",
    "2.responsibilities.7": "Authored reports, blogs, presentations, & custom researches in go-to-market strategy, deal
signing analysis, renewal analysis, buyer studies, technology adoptions (cloud, AI, ML, digital, etc.), and industry
trend analysis.",
    "3.responsibilities.0": "Delivered quarterly webinars on outsourcing/managed services signing trends.",
    "3.responsibilities.1": "Authored 7 to 10 pursuit strategy reports and industry trend research documents.",
    "4.responsibilities.0": "Designed and architected a complex company database with 100K+ unique records and
integrating DnB API and internal databases, which reduced data duplication by 50%.",
    "4.responsibilities.1": "Oversaw two Appian implementations that increased team productivity by 20 to 30%.",
    "4.responsibilities.2": "Managed daily content operations, including leading a team of offshore and nearshore
content team, as well as other sales and research related activities.",
    "4.responsibilities.3": "Analyzed and Modeled 20 to 30 IT vendors' financials and over 1,500 services contracts.",
    "4.responsibilities.4": "Managed three major data integration projects critical to the launch of the new platform.",
    "4.responsibilities.5": "Promoted from Senior Research Analyst to Research Manager in February 2007.",
    "5.responsibilities.0": "Researched market dynamics in the Web Services Security market to support product
strategy."
  }
}
```

### JSON format: edited by LLM, prioritized edited responsibilities manually

In [18]:
import xlwings as xw
import pandas as pd  # ✅ Required for pd.set_option and display
from IPython.display import display
from utils.get_file_names import get_file_names
from project_config import REVIEWED_MATCHINGS_DIR


# ✅ Get matching files
file_list = get_file_names(REVIEWED_MATCHINGS_DIR, full_path=True)

# ✅ Match company file
company = "salesforce"
company_file = next(
    (file for file in file_list if company.lower() in str(file).lower()), None
)

# ✅ Match company file
company = "salesforce"
company_file = next(
    (file for file in file_list if company.lower() in str(file).lower()), None
)

file_path = company_file
sheet_name = "Crosstab"

if file_path is not None:
    wb, sheet = load_excel_sheet(file_path, sheet_name)
    df = get_underlined_cells(sheet)
    cleaned_df = clean_and_remove_rows(df)
    final_df = rename_dataframe_columns(cleaned_df)
else:
    raise FileExistsError

final_df.drop("original_responsibility", axis=1, inplace=True)


# print(final_df)
json_string = final_df.to_json()
# json_data = json.loads(json_string)  # Convert string to dictionary

print(company)
print("Edited by LLM:")

# # Format JSON for readability
# formatted_json = format_json_readable(json_data, wrap_width=120)

# # Display with Markdown to prevent horizontal scrolling
# display(Markdown(f"```json\n{formatted_json}\n```"))


display_json_pretty(json_string)
wb.close()

salesforce
Edited by LLM:


```json
{
  "edited_responsibility_1": {
    "0.responsibilities.0": "Conversational AI & NLP: Passionately engineered the core dialogue system to dynamically
generate questions and evaluate responses, effectively solving customer issues through data-driven insights.",
    "0.responsibilities.1": "Designed AI-driven processes that generate, expand, cluster, and rank ideas, consistently
delivering actionable insights with measurable business outcomes.",
    "0.responsibilities.2": "Developed an AI system passionate about enhancing customer experiences by adapting
responses through analysis of user interactions and application of data-driven insights.",
    "0.responsibilities.3": "Built and executed a stateful tracking system to streamline topic transitions and enhance
collaboration among cross-functional teams, which resulted in more efficient and redundancy-free conversation
management.",
    "0.responsibilities.4": "Enhanced AI API performance for OpenAI GPT-4 & Anthropic Claude by specializing in
asynchronous integration and leveraging data-driven insights, committed to boosting user satisfaction through efficient
and effective resolution of customer issues.",
    "1.responsibilities.0": "Executed strategic research to enhance business environments, optimizing service partner
ecosystems for a major IT vendor in the Asia Pacific region.",
    "1.responsibilities.1": "Assisted a U.S.-based international services provider in its growth strategy by precisely
evaluating and scaling new engineering service opportunities in vital emerging markets. Demonstrated adeptness in
navigating complex organizational challenges through both collaborative and directive management styles.",
    "1.responsibilities.2": "Co-authored a comprehensive analysis on mergers and acquisitions in the engineering
services sector, focusing on transaction scales, skill gaps, and emerging opportunities. This research significantly
influenced strategic decisions regarding the integration of IT and operational technologies, reflecting a strong
commitment to solving customer challenges through insightful, data-driven strategies.",
    "1.responsibilities.3": "Enhanced data quality and consistency by collaborating closely with cross-functional vendor
teams, integrating detailed financial analysis, and standardizing methodologies.",
    "1.responsibilities.4": "Optimized resource allocation by achieving a significant centralization of tasks to an
offshore team in India, which enhanced executive-level resource management and demonstrated advanced presentation
skills.",
    "1.responsibilities.5": null,
    "1.responsibilities.6": "Collaborated with the engineering services research team to develop an innovative tracker
tool, authored pivotal analyses on market forecasts, the impact of COVID-19, and merger and acquisition trends,
demonstrating a strong passion for leveraging data to understand and address industry-specific challenges.",
    "1.responsibilities.7": "Executed and managed research projects across global business environments, collaborating
with analyst teams from regions including the US, Canada, Latin America, Europe, MEA, and APAC to enhance data quality
and ensure timely project completion. Applied significant expertise in research methodologies and tools effectively
within a business context.",
    "2.responsibilities.0": "Executed full P&L management, encompassing budgeting, HR, vendor relations, and
partnerships, demonstrating robust leadership and management skills. Successfully expanded program bookings by
strategically planning and delivering executive-level presentations.",
    "2.responsibilities.1": "Led and expanded a multinational research team, delivering data-driven insights with
measurable business impacts across the US, India, and Mexico, demonstrating a proven track record in insight work.",
    "2.responsibilities.2": "Led a team in the research and development of advanced software tools to enhance product
functionality and user experience throughout an extensive career spanning over a decade.",
    "2.responsibilities.3": "Championed numerous technology initiatives over the past decade at IDC, including
pioneering the first implementation of machine learning, chatbots, APIs, and ontologies. Directed comprehensive projects
involving ML, NLP, user experience, web scraping, and API integration, significantly advancing product development and
research.",
    "2.responsibilities.4": "Led an executive team in developing Python automation tools, enhancing operational
efficiency significantly and cutting report preparation time by 40%, demonstrating advanced executive-level presentation
skills.",
    "2.responsibilities.5": "Guided service firms through complex organizational challenges with collaborative and
directive management approaches.",
    "2.responsibilities.6": "Guided software vendors in devising services partnership strategies through expert
business-oriented research.",
    "2.responsibilities.7": "With extensive leadership in research spanning over a decade, the candidate has authored
numerous reports, blogs, and presentations focusing on go-to-market strategies, deal analysis, renewal trends, and buyer
behavior across various technology sectors including cloud, AI, ML, and digital transformations. These contributions
have significantly influenced product development and industry trend analysis within technological environments.",
    "3.responsibilities.0": "Led research and educational initiatives for over a decade, delivering quarterly webinars
on outsourcing and managed services trends, focusing on product development, user experience, and technology.",
    "3.responsibilities.1": "Authored multiple pursuit strategy reports and conducted industry trend analysis,
consistently delivering actionable insights that drove measurable business outcomes.",
    "4.responsibilities.0": "Designed and architected a complex company database by collaborating with cross-functional
teams, integrating external and internal data sources, which significantly reduced data duplication.",
    "4.responsibilities.1": "Managed multiple Appian implementations, successfully navigating complex organizational
challenges and boosting team productivity by 20-30% through collaborative and directive management styles.",
    "4.responsibilities.2": "Managed daily content operations effectively, led both offshore and nearshore teams, and
coordinated sales and research activities, including executive-level presentations.",
    "4.responsibilities.3": "Analyzed and modeled financial data across numerous IT vendors and service contracts,
demonstrating a collaborative mindset by effectively working with cross-functional teams.",
    "4.responsibilities.4": "Managed critical data integration projects essential for the successful launch of a new
platform and consistently demonstrated strong leadership and executive-level presentation skills.",
    "4.responsibilities.5": null,
    "5.responsibilities.0": "Executed market research in the Web Services Security sector to inform strategic business
decisions."
  }
}
```

### Final trim & edit by GPT, prioritized edited responsibilities manually

#### Create prompt

In [19]:
from prompts.prompt_templates import TRIM_CONDENSE_FINAL_RESUME_WITH_JOB_DESC_PROMPT
from models.resume_job_description_io_models import JobFileMappings
from evaluation_optimization.create_mapping_file import load_mappings_model_from_json

# Set file path and url
file_path = Path(sfdc_file)
url = sfdc_url


# * Fetch edited resume in json format
sheet_name = "Crosstab"
wb, sheet = load_excel_sheet(file_path, sheet_name)

df = get_underlined_cells(sheet)
cleaned_df = clean_and_remove_rows(df)
final_df = rename_dataframe_columns(cleaned_df)
final_df.drop("original_responsibility", axis=1, inplace=True)

resps_json = final_df.to_json()


# * Fetch job_description
job_descriptions_file = JOB_DESCRIPTIONS_JSON_FILE
job_descriptions_dict = load_and_clean_json_file(job_descriptions_file)
job_description = job_descriptions_dict[url]["data"]["content"]


# * Insert JSON into template
prompt_template = TRIM_CONDENSE_FINAL_RESUME_WITH_JOB_DESC_PROMPT
final_prompt = prompt_template.format(
    resume_json=resps_json, job_description_json=job_description
)

print(final_prompt)

2025-04-03 14:37:48,060 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\preprocessing\jobpostings.json



You are an expert resume optimizer for senior-level tech and consulting roles.
Your task is to **trim, consolidate, and optimize** the provided JSON resume data while ensuring alignment with the job description provided.

### **Task Instructions**
1. **Trim & Condense:** Reduce the resume word count to approximately **500-600 words** while retaining essential details.
2. **Merge Related Content:** Combine overlapping or redundant bullets to enhance readability and conciseness.
3. **Enhance ATS Compliance:** Ensure the use of **strong action verbs**, **industry-specific keywords**, and **quantifiable achievements** (e.g., "% improvements," "increased revenue by X%," "cut processing time by Y%").
4. **Align with Job Description:** Ensure the resume highlights **key skills, experience, and qualifications** relevant to the job description.
5. **Ensure Consistency & Clarity:** Maintain a **structured, professional format**, ensuring smooth readability while preserving key impact points.
6.

#### Create trimmed file name

In [20]:
from project_config import TRIMMED_MATCHINGS_DIR

file_path = Path(sfdc_file)
trimmed_file_name = f"{file_path.stem}_trimmed.json"
trimmed_file = TRIMMED_MATCHINGS_DIR / trimmed_file_name
print(trimmed_file_name)

Salesforce_com_Vice_President__Product_Research___Insights_crosstab_reviewed_trimmed.json


#### LLM response output - by OpenAI

In [21]:
file_path = trimmed_file
data = load_and_clean_json_file(file_path)

word_count = 0


# Iterate through the values of the JSON object
def count_words(data):
    word_count = 0
    for v in data.values():
        if isinstance(v, dict):
            word_count += count_words(v)
        elif isinstance(v, str):
            word_count += len(v.split())
    return word_count


word_count = count_words(data)


print("Trimmed and Condensed by GPT:")
print(f"Word count: {word_count}")
display_json_pretty(file_path)

2025-04-03 14:45:10,593 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\human_review\resps_reqs_matchings_trimmed\Salesforce_com_Vice_President__Product_Research___Insights_crosstab_reviewed_trimmed.json
2025-04-03 14:45:10,596 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\human_review\resps_reqs_matchings_trimmed\Salesforce_com_Vice_President__Product_Research___Insights_crosstab_reviewed_trimmed.json


Trimmed and Condensed by GPT:
Word count: 476


```json
{
  "edited_responsibility_1": {
    "0.responsibilities.0": "Engineered LLM-powered dialogue systems for structured user engagement and adaptive
response evaluation, improving resolution efficiency and customer satisfaction.",
    "0.responsibilities.1": "Designed AI pipelines for ideation and content workflows, leveraging clustering and ranking
methods to generate business-impactful insights.",
    "0.responsibilities.2": "Developed adaptive AI responses informed by real-time analysis of user input, enhancing
customer experience and resolution accuracy.",
    "0.responsibilities.3": "Built stateful tracking systems to manage topic transitions and minimize redundancy,
enabling smooth cross-functional collaboration and AI performance monitoring.",
    "0.responsibilities.4": "Optimized performance of OpenAI GPT-4 and Anthropic Claude through asynchronous API
integrations, boosting enterprise-scale responsiveness.",
    "1.responsibilities.0": "Delivered strategic research and insight to strengthen business environments, streamlining
service partner ecosystems for a major APAC IT vendor.",
    "1.responsibilities.1": "Assisted an international services provider in evaluating and scaling engineering services
in emerging markets, navigating complex org structures through collaborative and directive leadership.",
    "1.responsibilities.2": "Co-authored M&A research for the engineering services sector, providing data-driven
insights into transaction scales and IT/OT convergence that shaped executive decision-making.",
    "1.responsibilities.3": "Improved data quality through vendor collaboration and financial analysis, establishing
standardized research methodologies.",
    "1.responsibilities.4": "Centralized operations to offshore teams, improving executive-level resourcing and
elevating strategic presentations to leadership.",
    "1.responsibilities.6": "Developed a proprietary tracker tool; published research on service market forecasts, M&A,
and pandemic impact, showcasing strong analytical acumen and research leadership.",
    "1.responsibilities.7": "Led and coordinated global research projects, applying advanced research techniques across
APAC, EMEA, and the Americas to ensure high-quality, on-time deliverables.",
    "2.responsibilities.0": "Managed full P&L including budgeting, staffing, and partnerships; delivered >50% program
bookings growth through strategic planning and executive presentations.",
    "2.responsibilities.1": "Led multinational research teams in the US, India, and Mexico, producing actionable
insights that drove measurable outcomes in software and services markets.",
    "2.responsibilities.2": "Directed development of advanced research tools, driving UX improvements and enhancing
product functionality through ML and automation.",
    "2.responsibilities.3": "Spearheaded major ML, chatbot, ontology, and API initiatives, advancing enterprise
automation and customer intelligence tools.",
    "2.responsibilities.4": "Led executive Python automation initiatives that improved reporting workflows and cut prep
time by over 40%.",
    "2.responsibilities.5": "Advised service firms on managing organizational transformation using hybrid collaborative-
directive leadership models.",
    "2.responsibilities.6": "Guided software vendors in designing go-to-market service strategies via business-aligned
research and consulting.",
    "2.responsibilities.7": "Produced thought leadership on enterprise tech trends, including AI, cloud, and digital
transformation, influencing strategic roadmaps and deal strategies.",
    "3.responsibilities.0": "Delivered quarterly webinars and trend analyses on outsourcing and managed services,
advancing product and UX insights.",
    "3.responsibilities.1": "Authored pursuit strategy reports and trend research that translated into quantifiable
business results and market positioning improvements.",
    "4.responsibilities.0": "Architected a scalable enterprise database integrating external and internal sources,
reducing duplication and supporting business-critical reporting.",
    "4.responsibilities.1": "Managed Appian workflow implementations, boosting team productivity by 20–30% via effective
change leadership.",
    "4.responsibilities.2": "Oversaw offshore/nearshore operations, leading content, research, and sales support
activities with executive-level delivery.",
    "4.responsibilities.3": "Conducted financial modeling for 20–30 major IT vendors, aligning contract data with
strategic insights through collaborative vendor engagement.",
    "4.responsibilities.4": "Directed mission-critical data integration for a major platform launch, demonstrating
leadership and cross-team execution.",
    "5.responsibilities.0": "Led market research in Web Services Security to shape product direction, aligning
competitive intelligence with strategic planning."
  }
}
```

In [22]:
# In text copy/paste friendly format

file_path = trimmed_file
data = load_and_clean_json_file(file_path)

edited_resps = data["edited_responsibility_1"]

formatted_text = "\n\n".join(f"{key}\n{value}" for key, value in edited_resps.items())

print(formatted_text)

2025-04-03 14:46:23,658 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\human_review\resps_reqs_matchings_trimmed\Salesforce_com_Vice_President__Product_Research___Insights_crosstab_reviewed_trimmed.json


0.responsibilities.0
Engineered LLM-powered dialogue systems for structured user engagement and adaptive response evaluation, improving resolution efficiency and customer satisfaction.

0.responsibilities.1
Designed AI pipelines for ideation and content workflows, leveraging clustering and ranking methods to generate business-impactful insights.

0.responsibilities.2
Developed adaptive AI responses informed by real-time analysis of user input, enhancing customer experience and resolution accuracy.

0.responsibilities.3
Built stateful tracking systems to manage topic transitions and minimize redundancy, enabling smooth cross-functional collaboration and AI performance monitoring.

0.responsibilities.4
Optimized performance of OpenAI GPT-4 and Anthropic Claude through asynchronous API integrations, boosting enterprise-scale responsiveness.

1.responsibilities.0
Delivered strategic research and insight to strengthen business environments, streamlining service partner ecosystems for a majo

## Veeva Job Posting

### Get file name and url

In [27]:
import xlwings as xw
import pandas as pd
from IPython.display import display
from utils.get_file_names import get_file_names
from evaluation_optimization.create_mapping_file import load_mappings_model_from_json
from project_config import (
    ITERATE_1_OPENAI_DIR,
    mapping_file_name,
    REVIEWED_MATCHINGS_DIR,
)

# Get matching files
file_list = get_file_names(REVIEWED_MATCHINGS_DIR, full_path=True)

# Match company file (case-insensitive)
company = "Veeva"
company_file = next(
    (file for file in file_list if company.lower() in str(file).lower()), None
)

if not company_file:
    raise FileNotFoundError(f"No reviewed matchings file found for {company}")

# Load mapping file model
mapping_file = ITERATE_1_OPENAI_DIR / mapping_file_name
mapping_file_model = load_mappings_model_from_json(mapping_file)

# print(mapping_file_model)


# Get company URL(s) from mapping model
company_url = next(
    str(key) for key in mapping_file_model.root if company.lower() in str(key).lower()
)

if not company_url:
    print(f"⚠️ No URLs found for company '{company}' in mapping file.")

# Output
print(f"\n✅ {company} file:\n{company_file}")
print(f"\n🔗 {company} URL(s):{company_url}")

2025-03-26 09:58:22,376 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\evaluation_optimization\evaluation_optimization_by_openai\iteration_1\url_to_file_mapping.json
2025-03-26 09:58:22,378 - evaluation_optimization.create_mapping_file - INFO - Loaded and validated mapping file from C:\github\job_bot\input_output\evaluation_optimization\evaluation_optimization_by_openai\iteration_1\url_to_file_mapping.json



✅ Veeva file:
C:\github\job_bot\input_output\human_review\resps_reqs_matchings_reviewed\Veeva_Director_-_Crossix_Analytics_Services_crosstab_reviewed.xlsx

🔗 Veeva URL(s):https://careers.veeva.com/job/365ff44c-8e0a-42b4-a117-27b409a77753/director-crossix-analytics-services-boston-ma/?lever-source=Linkedin


In [36]:
# Set file and url vars

if company_file is None:
    raise ValueError("company_file (veeva_file) is None")

if company_url is None:
    raise ValueError("company_url (veeva_url) is None")

veeva_file = company_file
veeva_url = company_url

print(f"{company} file and url parameter set.\n{company_file}\n{company_url}")

Veeva file and url parameter set.
C:\github\job_bot\input_output\human_review\resps_reqs_matchings_reviewed\Veeva_Director_-_Crossix_Analytics_Services_crosstab_reviewed.xlsx
https://careers.veeva.com/job/365ff44c-8e0a-42b4-a117-27b409a77753/director-crossix-analytics-services-boston-ma/?lever-source=Linkedin


### Dataframe format, prioritized edited responsibilities manually

In [30]:
import xlwings as xw
import pandas as pd  # ✅ Required for pd.set_option and display
from IPython.display import display
from utils.get_file_names import get_file_names
from project_config import REVIEWED_MATCHINGS_DIR


# ✅ Get matching files
file_list = get_file_names(REVIEWED_MATCHINGS_DIR, full_path=True)

# ✅ Match company file
company = "Veeva"
veeva_file = next(
    (file for file in file_list if company.lower() in str(file).lower()), None
)

cross_tab_file = veeva_file

sheet_name = "Crosstab"

try:
    if not cross_tab_file:
        raise FileNotFoundError(f"No file found for company: {company}")

    df_all_data = pd.read_excel(cross_tab_file)

    wb, sheet = load_excel_sheet(file_path=cross_tab_file, sheet_name=sheet_name)
    df, coordinates = get_underlined_cells(sheet=sheet, return_coords=True)

except Exception as e:
    print(f"Error loading job posting: {e}")
    df = None


# df is your DataFrame
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

# ✅ Continue only if DataFrame loaded successfully
if df is not None:
    cleaned_df = clean_and_remove_rows(df)
    final_df = rename_dataframe_columns(cleaned_df)

    print(company)
    print(cross_tab_file)
    # pd.set_option("display.max_columns", None)
    display(final_df)

    wb.close()

pd.set_option("display.max_colwidth", 90)  # or a large number like 200

# pyright: ignore[reportInvalidTypeForm]
summary_df = summarize_underlined_column_frequencies(
    df_all_data=df_all_data,
    underlined_coords=coordinates,
)
print(f"Requirements Balance: \n{summary_df}")
pd.reset_option("all")

Veeva
C:\github\job_bot\input_output\human_review\resps_reqs_matchings_reviewed\Veeva_Director_-_Crossix_Analytics_Services_crosstab_reviewed.xlsx


,original_responsibility,edited_responsibility_1
responsibility_key,,
0.responsibilities.0,Conversational AI & NLP: Engineered the core d...,Designed and supported product development by ...
0.responsibilities.1,Thought Generation & Processing: Designed pipe...,Designed and implemented AI-driven systems tha...
0.responsibilities.2,Automated Evaluation & Adaptation: Developed a...,Developed an AI tool that proactively analyzes...
0.responsibilities.3,State & Topic Management: Built a stateful tra...,Built a stateful tracking system to enhance pr...
0.responsibilities.4,Asynchronous AI Integration: Optimized API per...,Asynchronous AI Integration: Led the optimizat...
1.responsibilities.0,Provided strategic insights to a major global ...,Led strategic analytics engagements for a majo...
1.responsibilities.1,Assisted a U.S.-based international services p...,Assisted a U.S.-based international services p...
1.responsibilities.2,Co-authored an industry-recognized report on M...,Conducted a thorough analysis of mergers and a...
1.responsibilities.3,Enhanced data quality and consistency by integ...,"Led consultative analytics engagements, enhanc..."


Requirements Balance: 
                                                                                 Column Name  Underline Count  Frequency (%)
1                         Experience leading consultative analytics engagements with clients                7           24.1
3                                                             8+ years in an analytical role                6           20.7
5                       Support product development efforts through client services feedback                5           17.2
0  Ability to use critical thinking to identify issues or concerns prior to them becoming...                4           13.8
4  Responsible for storytelling with data and strategic direction of your accounts, as we...                2            6.9
6  Be a leader on the team Coach, train, mentor in project and client service best practices                2            6.9
7                                                           Strong project management skills          

C:\Users\xzhan\AppData\Local\Temp\ipykernel_35600\4097763665.py:59: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option("all")
C:\Users\xzhan\AppData\Local\Temp\ipykernel_35600\4097763665.py:59: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option("all")


### JSON format: Original, prioritized edited responsibilities manually

In [33]:
file_path = veeva_file
sheet_name = "Crosstab"

if file_path is not None:
    wb, sheet = load_excel_sheet(file_path, sheet_name)
    df = get_underlined_cells(sheet)
    cleaned_df = clean_and_remove_rows(df)
    final_df = rename_dataframe_columns(cleaned_df)
else:
    raise FileExistsError

final_df.drop("edited_responsibility_1", axis=1, inplace=True)

json_string = final_df.to_json()
json_data = json.loads(json_string)  # Convert string to dictionary

print("JSON: original")
display_json_pretty(json_data)

wb.close()

JSON: original


```json
{
  "original_responsibility": {
    "0.responsibilities.0": "Conversational AI & NLP: Engineered the core dialogue system to generate dynamic questions,
evaluate responses, and maintain conversation flow.",
    "0.responsibilities.1": "Thought Generation & Processing: Designed pipelines for hierarchical idea expansion,
clustering, and ranking AI-generated thoughts.",
    "0.responsibilities.2": "Automated Evaluation & Adaptation: Developed an AI-based evaluator to assess user
responses, generate adaptive follow-ups, and track discussion depth.",
    "0.responsibilities.3": "State & Topic Management: Built a stateful tracking system to guide conversations, handle
topic transitions, and prevent redundancy.",
    "0.responsibilities.4": "Asynchronous AI Integration: Optimized API performance for OpenAI GPT-4 & Anthropic Claude
using AsyncIO, improving efficiency and scalability.",
    "1.responsibilities.0": "Provided strategic insights to a major global IT vendor, optimizing their service partner
ecosystem in Asia Pacific for improved local implementation outcomes.",
    "1.responsibilities.1": "Assisted a U.S.-based international services provider in its growth strategy by precisely
evaluating and scaling new engineering service opportunities in vital emerging markets.",
    "1.responsibilities.2": "Co-authored an industry-recognized report on M&A in the engineering services sector,
offering deep dives into deal sizes, capability gaps, and emerging opportunities, influencing strategic decisions in IT
and operational technology convergence.",
    "1.responsibilities.3": "Enhanced data quality and consistency by integrating thorough financial analysis,
standardizing methodologies, and conducting in-depth vendor engagements.",
    "1.responsibilities.4": "Achieved over 40% centralization of tasks to an offshore team in India, optimizing resource
allocation.",
    "1.responsibilities.5": "Developed Python tools to automate and accelerate internal processes, cutting report
preparation and data analysis time by over 40%.",
    "1.responsibilities.6": "Collaborated with the engineering services research team to pioneer the engineering
services tracker, authored influential publications on market forecasts, the impact of COVID-19 on services, and trends
in M&A within the engineering services industry.",
    "1.responsibilities.7": "Collaborated with analyst teams across the globe (US, Canada, Latin America, Europe, MEA,
APAC) to ensure data quality, meeting deadlines, sharing knowledge/best practices/methodology, and procure new tools.",
    "2.responsibilities.0": "Full P&L management including budgeting, HR, vendors, partnerships, research, and business
development: expanded the programs' bookings by more than 50%.",
    "2.responsibilities.1": "Expanded and managed a global research team of more than 15 (US, India, and Mexico).",
    "2.responsibilities.2": "Led an external software development team to build and implement new tools.",
    "2.responsibilities.3": "Championed new technology projects using ML, NLP, chatbot, ontology, web-scraping, API, UX
(User Experience). First in IDC to implement ML (machine learning). First to launch a chatbot. First to implement API
(application programming interface). First to build an ontology.",
    "2.responsibilities.4": "Led a team to develop automated tools in Python, reducing report preparation time by 40%.",
    "2.responsibilities.5": "Advised services firms on deal pursuit and sales orchestration strategies.",
    "2.responsibilities.6": "Advised software vendors on services partnership strategy.",
    "2.responsibilities.7": "Authored reports, blogs, presentations, & custom researches in go-to-market strategy, deal
signing analysis, renewal analysis, buyer studies, technology adoptions (cloud, AI, ML, digital, etc.), and industry
trend analysis.",
    "3.responsibilities.0": "Delivered quarterly webinars on outsourcing/managed services signing trends.",
    "3.responsibilities.1": "Authored 7 to 10 pursuit strategy reports and industry trend research documents.",
    "4.responsibilities.0": "Designed and architected a complex company database with 100K+ unique records and
integrating DnB API and internal databases, which reduced data duplication by 50%.",
    "4.responsibilities.1": "Oversaw two Appian implementations that increased team productivity by 20 to 30%.",
    "4.responsibilities.2": "Managed daily content operations, including leading a team of offshore and nearshore
content team, as well as other sales and research related activities.",
    "4.responsibilities.3": "Analyzed and Modeled 20 to 30 IT vendors' financials and over 1,500 services contracts.",
    "4.responsibilities.4": "Managed three major data integration projects critical to the launch of the new platform.",
    "4.responsibilities.5": "Promoted from Senior Research Analyst to Research Manager in February 2007.",
    "5.responsibilities.0": "Researched market dynamics in the Web Services Security market to support product
strategy."
  }
}
```

### JSON format: edited by LLM, prioritized edited responsibilities manually

In [35]:
file_path = veeva_file
sheet_name = "Crosstab"


wb, sheet = load_excel_sheet(file_path, sheet_name)

df = get_underlined_cells(sheet)
cleaned_df = clean_and_remove_rows(df)

final_df = rename_dataframe_columns(cleaned_df)
final_df.drop("original_responsibility", axis=1, inplace=True)


# print(final_df)
json_string = final_df.to_json()
# json_data = json.loads(json_string)  # Convert string to dictionary

print(company)
print("Edited by LLM:")

# # Format JSON for readability
# formatted_json = format_json_readable(json_data, wrap_width=120)

# # Display with Markdown to prevent horizontal scrolling
# display(Markdown(f"```json\n{formatted_json}\n```"))


display_json_pretty(json_string)
wb.close()

Veeva
Edited by LLM:


```json
{
  "edited_responsibility_1": {
    "0.responsibilities.0": "Designed and supported product development by engineering a conversational AI and NLP
system that generates dynamic questions, evaluates responses, and maintains conversation flow, integrating client
feedback to continuously refine functionalities.",
    "0.responsibilities.1": "Designed and implemented AI-driven systems that prioritize and process client-generated
ideas, leveraging customer feedback for product development.",
    "0.responsibilities.2": "Developed an AI tool that proactively analyzes critical metrics to identify and address
potential user engagement issues before escalation.",
    "0.responsibilities.3": "Built a stateful tracking system to enhance product development by guiding conversation
flows, managing client interactions, and capturing essential feedback.",
    "0.responsibilities.4": "Asynchronous AI Integration: Led the optimization of API performance for OpenAI GPT-4 &
Anthropic Claude using AsyncIO, enhancing efficiency and scalability, and catalyzing client analytics engagements.",
    "1.responsibilities.0": "Led strategic analytics engagements for a major global IT vendor, enhancing and optimizing
the service partner ecosystem across Asia Pacific for improved local implementation outcomes.",
    "1.responsibilities.1": "Assisted a U.S.-based international services provider in its strategic growth by focusing
on the scaling of engineering services in key emerging markets, utilizing data-driven storytelling and complex analytics
to make strategic decisions and direct account strategies.",
    "1.responsibilities.2": "Conducted a thorough analysis of mergers and acquisitions in the engineering sector,
employing critical thinking to explore deal dimensions, skill disparities, and emerging opportunities. This proactive
evaluation guided strategic decision-making, especially in merging IT with operational technologies.",
    "1.responsibilities.3": "Led consultative analytics engagements, enhancing client experiences through comprehensive
financial analysis, methodology standardization, and in-depth vendor interactions.",
    "1.responsibilities.4": "Achieved over 40% centralization of tasks to an offshore team in India, where I led,
coached, and mentored the team in project management and client service best practices.",
    "1.responsibilities.5": "Developed Python tools to automate and accelerate internal processes, enhancing critical
thinking skills by streamlining report preparation and data analysis, thus facilitating the proactive identification and
resolution of potential issues before escalation.",
    "1.responsibilities.6": "Collaborated with the engineering services research team to develop tools like the
engineering services tracker and generate strategic insights through data-driven storytelling. Authored key publications
that analyzed market forecasts, the impact of COVID-19 on the industry, and trends in mergers and acquisitions within
the engineering services sector, guiding the strategic direction for industry stakeholders and collaborating on complex
analytics.",
    "1.responsibilities.7": "Led global analytics teams, engaging directly with clients in regions including the US,
Canada, Latin America, Europe, MEA, and APAC, ensuring data quality assurance, meeting deadlines, and integrating new
analytics tools and methodologies.",
    "2.responsibilities.0": "Managed full P&L responsibilities encompassing budgeting, HR, vendor relations, and
partnerships, emphasizing leadership in business development and research. Executed consultative analytics engagements
with clients, achieving significant increases in program bookings.",
    "2.responsibilities.1": "Expanded and managed a multinational research team across the US, India, and Mexico,
focusing on roles demanding extensive analytical skills for more than a decade.",
    "2.responsibilities.2": "Led an external team in creating and launching innovative software solutions, leveraging
extensive experience in diverse analytical roles for more than a decade.",
    "2.responsibilities.3": "Pioneered technology initiatives that integrated advanced technologies like machine
learning, natural language processing, and APIs, enhancing product development and user experience, reflected in
positive client service feedback.",
    "2.responsibilities.4": "Led a team to develop automation tools that streamlined product development processes and
incorporated client feedback, enhancing service delivery.",
    "2.responsibilities.5": "Advised firms on client-facing advisory services, focusing on deal pursuit and sales
orchestration strategies, with significant experience spanning several years.",
    "2.responsibilities.6": "Advised software vendors on services partnership strategy, drawing on extensive experience
in analytical roles for over ten years.",
    "2.responsibilities.7": "As a leader, I authored reports, blogs, and presentations, targeting go-to-market
strategies, deal analysis, and technology trends in cloud computing, AI, and ML. I also coached and mentored team
members to enhance their capabilities in project execution and client service, strengthening our client engagement with
strategic industry insights.",
    "3.responsibilities.0": "Delivered quarterly webinars on outsourcing and managed services signing trends,
demonstrating an extensive career in analytical roles spanning nearly a decade.",
    "3.responsibilities.1": "Demonstrates robust project management abilities by authoring numerous pursuit strategy
reports and executing extensive industry trend research.",
    "4.responsibilities.0": "Designed and architected a complex company database, integrating external and internal data
sources, which significantly enhanced data efficiency and improved client consultations, demonstrating leadership in
consultative analytics engagements.",
    "4.responsibilities.1": "Oversaw Appian implementations, proactively identified and resolved potential issues,
boosting team productivity by 20-30%.",
    "4.responsibilities.2": "Managed daily content operations and led a team comprising offshore and nearshore members,
while actively engaging in consultative analytics with clients.",
    "4.responsibilities.3": "Specialized in financial analysis and modeling, analyzing and modeling financials for
numerous IT vendors and their extensive service contracts for over eight years.",
    "4.responsibilities.4": "Led multiple key data integration projects, demonstrating strong project management skills
essential for the successful launch of a new platform.",
    "4.responsibilities.5": null,
    "5.responsibilities.0": "Analyzed market dynamics in the Web Services Security sector for over eight years, steering
product strategy."
  }
}
```

### Final trim & edit by GPT, prioritized edited responsibilities manually

#### Create prompt

In [37]:
from prompts.prompt_templates import TRIM_CONDENSE_FINAL_RESUME_WITH_JOB_DESC_PROMPT
from models.resume_job_description_io_models import JobFileMappings
from evaluation_optimization.create_mapping_file import load_mappings_model_from_json

# Set file path and url
file_path = Path(veeva_file)
url = veeva_url


# * Fetch edited resume in json format
sheet_name = "Crosstab"
wb, sheet = load_excel_sheet(file_path, sheet_name)

df = get_underlined_cells(sheet)
cleaned_df = clean_and_remove_rows(df)
final_df = rename_dataframe_columns(cleaned_df)
final_df.drop("original_responsibility", axis=1, inplace=True)

resps_json = final_df.to_json()


# * Fetch job_description
job_descriptions_file = JOB_DESCRIPTIONS_JSON_FILE
job_descriptions_dict = load_and_clean_json_file(job_descriptions_file)
job_description = job_descriptions_dict[url]["data"]["content"]


# * Insert JSON into template
prompt_template = TRIM_CONDENSE_FINAL_RESUME_WITH_JOB_DESC_PROMPT
final_prompt = prompt_template.format(
    resume_json=resps_json, job_description_json=job_description
)

print(final_prompt)

2025-03-26 10:10:46,764 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\preprocessing\jobpostings.json



You are an expert resume optimizer for senior-level tech and consulting roles.
Your task is to **trim, consolidate, and optimize** the provided JSON resume data while ensuring alignment with the job description provided.

### **Task Instructions**
1. **Trim & Condense:** Reduce the resume word count to approximately **500-600 words** while retaining essential details.
2. **Merge Related Content:** Combine overlapping or redundant bullets to enhance readability and conciseness.
3. **Enhance ATS Compliance:** Ensure the use of **strong action verbs**, **industry-specific keywords**, and **quantifiable achievements** (e.g., "% improvements," "increased revenue by X%," "cut processing time by Y%").
4. **Align with Job Description:** Ensure the resume highlights **key skills, experience, and qualifications** relevant to the job description.
5. **Ensure Consistency & Clarity:** Maintain a **structured, professional format**, ensuring smooth readability while preserving key impact points.
6.

#### Create trimmed file name

In [38]:
from project_config import TRIMMED_MATCHINGS_DIR

file_path = Path(veeva_file)
trimmed_file_name = f"{file_path.stem}_trimmed.json"
trimmed_file = TRIMMED_MATCHINGS_DIR / trimmed_file_name
print(trimmed_file_name)

Veeva_Director_-_Crossix_Analytics_Services_crosstab_reviewed_trimmed.json


#### LLM response output - by OpenAI

In [40]:
file_path = trimmed_file
data = load_and_clean_json_file(file_path)

word_count = 0


# Iterate through the values of the JSON object
def count_words(data):
    word_count = 0
    for v in data.values():
        if isinstance(v, dict):
            word_count += count_words(v)
        elif isinstance(v, str):
            word_count += len(v.split())
    return word_count


word_count = count_words(data)


print("Trimmed and Condensed by GPT:")
print(f"Word count: {word_count}")
display_json_pretty(file_path)

2025-03-26 10:20:51,662 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\human_review\resps_reqs_matchings_trimmed\Veeva_Director_-_Crossix_Analytics_Services_crosstab_reviewed_trimmed.json
2025-03-26 10:20:51,664 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\human_review\resps_reqs_matchings_trimmed\Veeva_Director_-_Crossix_Analytics_Services_crosstab_reviewed_trimmed.json


Trimmed and Condensed by GPT:
Word count: 466


```json
{
  "edited_responsibility_1": {
    "0.responsibilities.0": "Engineered a multi-agent conversational AI system that dynamically generated questions,
evaluated responses, and maintained topic coherence—enhancing user engagement and client analytics.",
    "0.responsibilities.1": "Built AI systems for ideation, prioritizing client-generated ideas and leveraging NLP to
guide product development.",
    "0.responsibilities.2": "Developed proactive analytics tools to flag and resolve user engagement issues before
escalation, improving product performance.",
    "0.responsibilities.3": "Designed a stateful conversation tracking system to manage interactions and capture user
feedback for iterative improvements.",
    "0.responsibilities.4": "Boosted AI system scalability by optimizing OpenAI GPT-4 & Claude API calls using
AsyncIO—enabling faster, asynchronous LLM processing.",
    "1.responsibilities.0": "Led analytics initiatives for a global IT vendor to optimize its Asia Pacific service
partner network—enhancing local market execution.",
    "1.responsibilities.1": "Advised an international services provider on growth strategies, using data-driven analysis
to scale engineering services in key markets.",
    "1.responsibilities.2": "Evaluated M&A activity in engineering services to identify deal trends and convergence
opportunities across IT and OT domains.",
    "1.responsibilities.3": "Enhanced client strategy through financial modeling, standardized methodologies, and
collaborative vendor engagement.",
    "1.responsibilities.4": "Achieved 40% task centralization to an offshore team in India by coaching team members on
project and client service delivery.",
    "1.responsibilities.5": "Automated reporting workflows using Python, reducing prep time and surfacing insights to
preempt client pain points.",
    "1.responsibilities.6": "Developed strategic research tools including a services tracker and authored insights on
COVID-19 impact, forecasts, and M&A trends.",
    "1.responsibilities.7": "Managed global analytics teams across US, LATAM, Europe, MEA, and APAC—ensuring quality,
timeliness, and client integration.",
    "2.responsibilities.0": "Oversaw end-to-end P&L for services analytics program—growing bookings by over 50% through
business development and research innovation.",
    "2.responsibilities.1": "Led and expanded a cross-functional, multinational team across US, India, and Mexico with
strong analytical delivery capabilities.",
    "2.responsibilities.2": "Managed external software development for analytics tools, applying over a decade of
analytical leadership experience.",
    "2.responsibilities.3": "Pioneered ML, NLP, and API-based innovations to enhance product capabilities and improve
client satisfaction.",
    "2.responsibilities.4": "Built automation tools and feedback loops to streamline product development and elevate
service responsiveness.",
    "2.responsibilities.5": "Provided advisory support to firms on deal pursuit and sales orchestration, leveraging
market insights and long-term experience.",
    "2.responsibilities.6": "Guided software vendors on partner ecosystem strategy, anchored in a decade of analytical
and market strategy expertise.",
    "2.responsibilities.7": "Authored thought leadership on go-to-market, deal strategy, and AI trends; mentored team
members to build client-ready capabilities.",
    "3.responsibilities.0": "Delivered executive webinars on outsourcing and managed services trends—supporting C-suite
decision-making across industries.",
    "3.responsibilities.1": "Produced strategic pursuit reports and industry research to drive market positioning and
project execution excellence.",
    "4.responsibilities.0": "Designed a scalable enterprise database integrating internal and DnB data—enhancing client
consulting and data accuracy.",
    "4.responsibilities.1": "Directed Appian workflow deployments that boosted team productivity by 20–30%, improving
operational delivery.",
    "4.responsibilities.2": "Led daily content operations and collaborated with offshore/nearshore teams while
supporting consultative analytics delivery.",
    "4.responsibilities.3": "Conducted financial modeling on 20–30 IT vendors and 1,500+ service contracts—delivering
strategic insights for industry clients.",
    "4.responsibilities.4": "Led data integration initiatives for platform rollouts, showcasing strong cross-functional
project management expertise.",
    "5.responsibilities.0": "Conducted market analysis on Web Services Security for product strategy support—laying
foundation for future enterprise integration."
  }
}
```

In [43]:
# In text copy/paste friendly format

file_path = trimmed_file
data = load_and_clean_json_file(file_path)

edited_resps = data["edited_responsibility_1"]

formatted_text = "\n\n".join(f"{key}\n{value}" for key, value in edited_resps.items())

print(formatted_text)

2025-03-26 10:53:41,368 - utils.generic_utils - INFO - Loaded data from C:\github\job_bot\input_output\human_review\resps_reqs_matchings_trimmed\Veeva_Director_-_Crossix_Analytics_Services_crosstab_reviewed_trimmed.json


0.responsibilities.0
Engineered a multi-agent conversational AI system that dynamically generated questions, evaluated responses, and maintained topic coherence—enhancing user engagement and client analytics.

0.responsibilities.1
Built AI systems for ideation, prioritizing client-generated ideas and leveraging NLP to guide product development.

0.responsibilities.2
Developed proactive analytics tools to flag and resolve user engagement issues before escalation, improving product performance.

0.responsibilities.3
Designed a stateful conversation tracking system to manage interactions and capture user feedback for iterative improvements.

0.responsibilities.4
Boosted AI system scalability by optimizing OpenAI GPT-4 & Claude API calls using AsyncIO—enabling faster, asynchronous LLM processing.

1.responsibilities.0
Led analytics initiatives for a global IT vendor to optimize its Asia Pacific service partner network—enhancing local market execution.

1.responsibilities.1
Advised an inter

# End